## prediksi tanpa clustering (K-fold crossval)

In [1]:
import numpy as np
import pandas as pd

from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:

# Konfigurasi kolom

FILE_CLUSTERED = "D:/DATA SKRIPSI/kontrak_sewa_bersih.xlsx"
CLUSTER_COL    = "cluster"        
PRICE_COL      = "CuryUnitPrice"  

CATEGORICAL_COLS = [
    "BusinessType","LeaseMonthStart","LeaseDayStart",
    "LeaseMonthEnd","LeaseDayEnd","TranCode",
    "ContractPeriod","ContractType","Building", "UnitArea", "UnitFloor"
]

NUMERIC_COLS = [
    "BuildingArea","LeaseDurationDays","LeaseDurationMonths",
    "LeaseYearStart","LeaseYearEnd","n_subunit"
]

# Konfigurasi per periode 
SHEET_CONFIGS = [
    {
        "sheet_cluster": "Monthly_Fixed_clustered",
        "sheet_name_out": "Monthly",      
    },
    {
        "sheet_cluster": "Daily_Fixed_clustered",
        "sheet_name_out": "Daily",        
    },
]

OUTPUT_FILE = "catboost_oof_pred_without_cluster_feature.xlsx"


# Fungsi

def clean_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = df.columns.astype(str).str.strip()
    return df


def run_for_sheet(
    file_path: str,
    sheet_cluster: str,
    sheet_name_out: str,
    writer: pd.ExcelWriter,
):
    """
    Jalankan CatBoost + 5-fold CV untuk satu sheet *_Fixed_clustered
    (tanpa menggunakan cluster sebagai fitur), lalu simpan OOF
    (cluster, harga aktual, prediksi) ke sheet Excel.
    """
    print("\n===================================================")
    print(f"Proses sheet cluster : {sheet_cluster}")
    print(f"Output Excel sheet   : {sheet_name_out}")
    print("=============================================\n")

    
    # Load sheet utama 
    
    try:
        df_main = clean_columns(pd.read_excel(file_path, sheet_name=sheet_cluster))
    except ValueError:
        df_main = clean_columns(pd.read_excel(file_path, sheet_name=0))

    
    if CLUSTER_COL not in df_main.columns:
        raise KeyError(
            f"Kolom cluster '{CLUSTER_COL}' tidak ditemukan di sheet '{sheet_cluster}'. "
            "Pastikan hasil clustering sudah disimpan sebagai kolom tersebut."
        )

    
    price_col = PRICE_COL
    if price_col not in df_main.columns:
        cands = [c for c in df_main.columns if "price" in c.lower()]
        if not cands:
            raise KeyError(
                f"Kolom target '{PRICE_COL}' tidak ditemukan dan tidak ada kolom mirip 'price' di sheet '{sheet_cluster}'."
            )
        price_col = cands[0]
        print(f"[INFO] Kolom {PRICE_COL} tidak ditemukan, pakai kolom mirip: {price_col}")

   
    y_all = (
        pd.to_numeric(df_main[price_col], errors="coerce")
        .astype(float)
    )

    cat_all = [c for c in CATEGORICAL_COLS if c in df_main.columns]
    num_all = [c for c in NUMERIC_COLS if c in df_main.columns]
    feature_cols = cat_all + num_all   

    X_all = df_main[feature_cols].copy()

    # kategorikal -> string
    for c in cat_all:
        X_all[c] = X_all[c].astype(str)

    # numerik -> float
    for c in num_all:
        X_all[c] = pd.to_numeric(X_all[c], errors="coerce").astype(float)

    keep_mask = ~y_all.isna()
    X = X_all.loc[keep_mask].reset_index(drop=True)
    y = y_all.loc[keep_mask].reset_index(drop=True)

    # index kolom kategorikal untuk CatBoost
    cat_feature_idx = [X.columns.get_loc(c) for c in cat_all]

    
    # 5-Fold Cross Validation dengan CatBoost
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    mae_scores, rmse_scores, r2_scores, rpmse_scores = [], [], [], []

    # oof_pred untuk seluruh df_main (index asli)
    oof_pred = np.full(len(df_main), np.nan)
    idx_map  = np.where(keep_mask)[0]

    for fold, (tr_idx, va_idx) in enumerate(kf.split(X), 1):
        X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
        y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

        train_pool = Pool(X_tr, y_tr, cat_features=cat_feature_idx)
        valid_pool = Pool(X_va, y_va, cat_features=cat_feature_idx)

        model = CatBoostRegressor(
            loss_function="RMSE",
            depth=8,
            learning_rate=0.05,
            l2_leaf_reg=3.0,
            iterations=2000,
            random_seed=42,
            od_type="Iter",
            od_wait=100,
            verbose=False
        )
        model.fit(train_pool, eval_set=valid_pool, use_best_model=True)

        pred_va = model.predict(valid_pool)
        oof_pred[idx_map[va_idx]] = pred_va

        # metrik
        mae   = mean_absolute_error(y_va, pred_va)
        rmse  = np.sqrt(mean_squared_error(y_va, pred_va))
        r2    = r2_score(y_va, pred_va)
        rpmse = (rmse / np.mean(y_va)) * 100  # RPMSE (%)

        mae_scores.append(mae)
        rmse_scores.append(rmse)
        r2_scores.append(r2)
        rpmse_scores.append(rpmse)

        print(
            f"[{sheet_cluster}] Fold {fold}: "
            f"MAE={mae:.2f} | RMSE={rmse:.2f} | RPMSE={rpmse:.2f}% | R²={r2:.3f}"
        )

    print(f"\n=== Hasil 5-Fold Cross-Validation ({sheet_cluster}) ===")
    print(f"MAE   : {np.mean(mae_scores):.2f} ± {np.std(mae_scores):.2f}")
    print(f"RMSE  : {np.mean(rmse_scores):.2f} ± {np.std(rmse_scores):.2f}")
    print(f"RPMSE : {np.mean(rpmse_scores):.2f}% ± {np.std(rpmse_scores):.2f}%")
    print(f"R²    : {np.mean(r2_scores):.3f} ± {np.std(r2_scores):.3f}")

    
    # Simpan hasil OOF 
    
    df_out = df_main.copy()
    df_out["oof_pred"] = oof_pred

    if price_col not in df_out.columns:
        df_out[price_col] = y_all.values

    cols_to_save = [CLUSTER_COL, price_col, "oof_pred"]
    df_out[cols_to_save].to_excel(
        writer,
        sheet_name=sheet_name_out,
        index=False
    )

    print(f"\nPrediksi OOF tersimpan di file: {OUTPUT_FILE} | sheet: {sheet_name_out}")

if __name__ == "__main__":
    with pd.ExcelWriter(OUTPUT_FILE) as writer:
        for cfg in SHEET_CONFIGS:
            run_for_sheet(
                file_path=FILE_CLUSTERED,
                sheet_cluster=cfg["sheet_cluster"],
                sheet_name_out=cfg["sheet_name_out"],
                writer=writer,
            )

    print(f"\nSemua hasil OOF tersimpan di file: {OUTPUT_FILE}")



Proses sheet cluster : Monthly_Fixed_clustered
Output Excel sheet   : Monthly

[Monthly_Fixed_clustered] Fold 1: MAE=73139.59 | RMSE=121950.09 | RPMSE=51.37% | R²=0.715
[Monthly_Fixed_clustered] Fold 2: MAE=70907.72 | RMSE=119148.29 | RPMSE=48.16% | R²=0.770
[Monthly_Fixed_clustered] Fold 3: MAE=77135.84 | RMSE=125754.90 | RPMSE=50.65% | R²=0.777
[Monthly_Fixed_clustered] Fold 4: MAE=77756.11 | RMSE=135166.34 | RPMSE=55.52% | R²=0.784
[Monthly_Fixed_clustered] Fold 5: MAE=76162.87 | RMSE=126805.85 | RPMSE=52.13% | R²=0.685

=== Hasil 5-Fold Cross-Validation (Monthly_Fixed_clustered) ===
MAE   : 75020.43 ± 2596.77
RMSE  : 125765.10 ± 5436.97
RPMSE : 51.56% ± 2.38%
R²    : 0.746 ± 0.039

Prediksi OOF tersimpan di file: catboost_oof_pred_without_cluster_feature.xlsx | sheet: Monthly

Proses sheet cluster : Daily_Fixed_clustered
Output Excel sheet   : Daily

[Daily_Fixed_clustered] Fold 1: MAE=6259.37 | RMSE=9583.06 | RPMSE=28.90% | R²=0.799
[Daily_Fixed_clustered] Fold 2: MAE=6514.28 | R

## Prediksi pakai walk forwad CV

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from catboost import CatBoostRegressor, Pool

def parse_num_id(value):
   
    if pd.isna(value):
        return np.nan

    if isinstance(value, (int, float, np.integer, np.floating)):
        return float(value)

    s = str(value).strip()
    if s == "":
        return np.nan

    s = s.replace(" ", "")

    if "," in s:
        left, right = s.split(",", 1)
        left = left.replace(".", "")
        s_clean = left + "." + right
        try:
            return float(s_clean)
        except ValueError:
            return np.nan
 
    else:
        s_clean = s.replace(".", "")
        try:
            return float(s_clean)
        except ValueError:
            return np.nan


def train_catboost_for_sheet(FILE_PATH, SHEET_NAME, model_suffix):
    print("\n" + "=" * 80)
    print(f"=== PROSES SHEET: {SHEET_NAME} ===")

    df = pd.read_excel(FILE_PATH, sheet_name=SHEET_NAME)

    print("Kolom di file:", list(df.columns))

    # parse kolom numerik
    numeric_cols = [
        'BuildingArea',
        'CuryUnitPrice',
        'LeaseDuration',
        'DurationMonth',
        'LeaseDurationDays',
        'LeaseDurationMonths',
        'n_subunit'
    ]

    for col in numeric_cols:
        if col in df.columns:
            df[col] = df[col].apply(parse_num_id)

    TARGET_COL = 'CuryUnitPrice'
    if TARGET_COL not in df.columns:
        raise ValueError(f"Kolom '{TARGET_COL}' (target) tidak ditemukan di sheet {SHEET_NAME}.")

    print("\nDeskripsi CuryUnitPrice (setelah parsing):")
    print(df[TARGET_COL].describe())

    # urutkan tanggal

    # Tanggal mulai sewa
    if {'LeaseYearStart', 'LeaseMonthStart', 'LeaseDayStart'}.issubset(df.columns):
        df['LeaseStartDate'] = pd.to_datetime(
            dict(
                year=df['LeaseYearStart'],
                month=df['LeaseMonthStart'],
                day=df['LeaseDayStart']
            ),
            errors='coerce'
        )
    else:
        raise ValueError(
            f"Kolom LeaseYearStart/LeaseMonthStart/LeaseDayStart tidak lengkap di sheet {SHEET_NAME}."
        )

    # Tanggal akhir sewa
    if {'LeaseYearEnd', 'LeaseMonthEnd', 'LeaseDayEnd'}.issubset(df.columns):
        df['LeaseEndDate'] = pd.to_datetime(
            dict(
                year=df['LeaseYearEnd'],
                month=df['LeaseMonthEnd'],
                day=df['LeaseDayEnd']
            ),
            errors='coerce'
        )
    else:
        df['LeaseEndDate'] = pd.NaT

    df = df.dropna(subset=['LeaseStartDate'])
    df = df.sort_values('LeaseStartDate').reset_index(drop=True)

    print("\nRange tanggal sewa:", df['LeaseStartDate'].min(), "->", df['LeaseStartDate'].max())


    df = df.dropna(subset=[TARGET_COL])

    # --- fitur tanggal dasar ---
    df['Year'] = df['LeaseStartDate'].dt.year
    df['Month'] = df['LeaseStartDate'].dt.month
    df['Quarter'] = df['LeaseStartDate'].dt.quarter

    df['YearMonth'] = df['LeaseStartDate'].dt.to_period('M')
    df['TrendIndex'] = (df['YearMonth'] - df['YearMonth'].min()).apply(lambda x: x.n)

    df['Month_sin'] = np.sin(2 * np.pi * df['Month'] / 12)
    df['Month_cos'] = np.cos(2 * np.pi * df['Month'] / 12)

    df = df.sort_values('LeaseStartDate').reset_index(drop=True)

    
    df['global_mean_target'] = df[TARGET_COL].expanding().mean().shift(1)
    overall_mean = df[TARGET_COL].mean()
    df['global_mean_target'] = df['global_mean_target'].fillna(overall_mean)

    # Lag & rolling per Building 
    if 'Building' in df.columns:
        g_bld = df.groupby('Building', group_keys=False)
        df['lag1_bld'] = g_bld[TARGET_COL].shift(1)
        df['lag3_bld'] = g_bld[TARGET_COL].shift(3)
        df['lag6_bld'] = g_bld[TARGET_COL].shift(6)
        df['roll3_bld'] = g_bld[TARGET_COL].shift(1).rolling(window=3, min_periods=1).mean()
        df['roll6_bld'] = g_bld[TARGET_COL].shift(1).rolling(window=6, min_periods=1).mean()
    else:
        df['lag1_bld'] = df['lag3_bld'] = df['lag6_bld'] = np.nan
        df['roll3_bld'] = df['roll6_bld'] = np.nan

    if {'Building', 'BusinessType'}.issubset(df.columns):
        g_bb = df.groupby(['Building', 'BusinessType'], group_keys=False)
        df['lag1_bld_bus'] = g_bb[TARGET_COL].shift(1)
        df['roll3_bld_bus'] = g_bb[TARGET_COL].shift(1).rolling(window=3, min_periods=1).mean()
    else:
        df['lag1_bld_bus'] = df['roll3_bld_bus'] = np.nan

    for col in [
        'lag1_bld', 'lag3_bld', 'lag6_bld',
        'roll3_bld', 'roll6_bld',
        'lag1_bld_bus', 'roll3_bld_bus'
    ]:
        df[col] = df[col].fillna(df['global_mean_target'])

    # siapkan fitur numerik & kategorik
    drop_cols = [TARGET_COL, 'LeaseStartDate', 'LeaseEndDate', 'YearMonth']
    drop_cols = [c for c in drop_cols if c in df.columns]

    feature_cols = [c for c in df.columns if c not in drop_cols]

    remove_features = ['UnitID', 'UnitNum']
    feature_cols = [c for c in feature_cols if c not in remove_features]

    X = df[feature_cols].copy()
    y_raw = df[TARGET_COL].copy()
    y = np.log1p(y_raw)           

    # Kategori eksplisit
    explicit_cat = [
        'ContractType',
        'ContractPeriod',
        'BusinessType',
        'TranCode',
        'Building',
        'UnitArea',
        'UnitFloor'
    ]
    explicit_cat = [c for c in explicit_cat if c in X.columns]
    auto_cat = X.select_dtypes(include=['object']).columns.tolist()

    cat_cols = sorted(list(set(explicit_cat) | set(auto_cat)))

    for c in cat_cols:
        X[c] = X[c].astype(str)

    cat_idx = [X.columns.get_loc(c) for c in cat_cols]

    print("\nJumlah data :", len(df))
    print("Jumlah fitur:", len(feature_cols))
    print("Fitur kategorik:", cat_cols)

    # catboost + walkforwardCV

    def rpmse(y_true, y_pred):
        y_true = np.asarray(y_true)
        y_pred = np.asarray(y_pred)
        rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
        return rmse / np.mean(y_true) * 100.0, rmse

    n_splits = 5
    tscv = TimeSeriesSplit(n_splits=n_splits)

    fold_results = []
    oof_pred = np.zeros(len(df))
    fold_id = np.zeros(len(df), dtype=int)  

    for fold, (train_idx, val_idx) in enumerate(tscv.split(X, y), start=1):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train_log, y_val_log = y.iloc[train_idx], y.iloc[val_idx]
        y_val_true = y_raw.iloc[val_idx]  

        train_pool = Pool(X_train, y_train_log, cat_features=cat_idx)
        val_pool = Pool(X_val, y_val_log, cat_features=cat_idx)

        model = CatBoostRegressor(
            loss_function='RMSE',
            depth=6,
            learning_rate=0.03,
            n_estimators=3000,
            random_seed=42,
            l2_leaf_reg=8,
            random_strength=1.5,
            bagging_temperature=1.0,
            eval_metric='RMSE',
            od_type='Iter',
            od_wait=100,
            verbose=False
        )

        model.fit(train_pool, eval_set=val_pool, use_best_model=True)

        y_val_pred_log = model.predict(X_val)
        y_val_pred = np.expm1(y_val_pred_log)  
        oof_pred[val_idx] = y_val_pred
        fold_id[val_idx] = fold  

        mae = mean_absolute_error(y_val_true, y_val_pred)
        rp, rmse = rpmse(y_val_true, y_val_pred)
        r2 = r2_score(y_val_true, y_val_pred)

        fold_results.append((mae, rmse, rp, r2))

        print(f"Fold {fold}: MAE={mae:,.2f} | RMSE={rmse:,.2f} | RPMSE={rp:.2f}% | R²={r2:.3f}")

    mae_list, rmse_list, rp_list, r2_list = map(np.array, zip(*fold_results))

    print(f"\n=== Hasil Walk-Forward CV ({SHEET_NAME}, LOG TARGET) ===")
    print(f"MAE   : {mae_list.mean():,.2f} ± {mae_list.std():,.2f}")
    print(f"RMSE  : {rmse_list.mean():,.2f} ± {rmse_list.std():,.2f}")
    print(f"RPMSE : {rp_list.mean():.2f}% ± {rp_list.std():.2f}%")
    print(f"R²    : {r2_list.mean():.3f} ± {r2_list.std():.3f}")

    # train model final
    final_pool = Pool(X, y, cat_features=cat_idx)
    final_model = CatBoostRegressor(
        loss_function='RMSE',
        depth=6,
        learning_rate=0.03,
        n_estimators=3000,
        random_seed=42,
        l2_leaf_reg=8,
        random_strength=1.5,
        bagging_temperature=1.0,
        eval_metric='RMSE',
        od_type='Iter',
        od_wait=100,
        verbose=False
    )
    final_model.fit(final_pool)

    print(f"\nModel final untuk sheet '{SHEET_NAME}' telah dilatih (tidak disimpan ke file).")
    print("Catatan: model memprediksi log1p(CuryUnitPrice), jadi saat inferensi perlu np.expm1(pred).")

    # simpan hasil prediksi

    id_cols = [
        'LeaseStartDate', 'LeaseEndDate',
        'Building', 'UnitID', 'UnitNum',
        'BusinessType', 'ContractType', 'ContractPeriod'
    ]
    id_cols = [c for c in id_cols if c in df.columns]

    oof_df = df[id_cols].copy()

    oof_df['CuryUnitPrice'] = y_raw.values      
    oof_df['oof_pred'] = oof_pred              
    oof_df['fold'] = fold_id                   
    oof_df['abs_error'] = (oof_df['CuryUnitPrice'] - oof_df['oof_pred']).abs()
    oof_df['ape'] = oof_df['abs_error'] / oof_df['CuryUnitPrice'] * 100.0

    
    print("\nPreview OOF DF untuk sheet:", SHEET_NAME)
    print(oof_df.head())
    print("Kolom OOF DF:", list(oof_df.columns))

    return oof_df  

if __name__ == "__main__":
    FILE_PATH = "D:/DATA SKRIPSI/kontrak_sewa_bersih.xlsx"  

    
    sheets_to_run = {
        "Monthly_Fixed": "monthly_sheet",
        "Daily_Fixed": "daily_sheet"
    }

    all_oof = {}

    for sheet_name, suffix in sheets_to_run.items():
        oof_df = train_catboost_for_sheet(FILE_PATH, sheet_name, suffix)
        all_oof[sheet_name] = oof_df

    output_excel = "hasil_catboost_oof_pred.xlsx"

    with pd.ExcelWriter(output_excel) as writer:
        for sheet_name, oof_df in all_oof.items():
            oof_df.to_excel(writer, sheet_name=sheet_name, index=False)

    print(f"\nSemua OOF prediction tersimpan di file Excel: {output_excel}")



=== PROSES SHEET: Monthly_Fixed ===
Kolom di file: ['RowID', 'ContractType', 'ContractPeriod', 'BusinessType', 'TranCode', 'BuildingArea', 'CuryUnitPrice', 'LeaseYearStart', 'LeaseMonthStart', 'LeaseDayStart', 'LeaseYearEnd', 'LeaseMonthEnd', 'LeaseDayEnd', 'LeaseDurationDays', 'LeaseDurationMonths', 'n_subunit', 'Building', 'UnitArea', 'UnitFloor', 'UnitNum', 'UnitID']

Deskripsi CuryUnitPrice (setelah parsing):
count    3.914000e+03
mean     2.439664e+05
std      2.531769e+05
min      9.000000e+03
25%      6.750000e+04
50%      1.750000e+05
75%      3.000000e+05
max      2.444444e+06
Name: CuryUnitPrice, dtype: float64

Range tanggal sewa: 2015-01-01 00:00:00 -> 2025-04-23 00:00:00

Jumlah data : 3914
Jumlah fitur: 32
Fitur kategorik: ['Building', 'BusinessType', 'ContractPeriod', 'ContractType', 'TranCode', 'UnitArea', 'UnitFloor']
Fold 1: MAE=83,749.57 | RMSE=159,351.99 | RPMSE=62.96% | R²=0.510
Fold 2: MAE=85,195.63 | RMSE=150,598.46 | RPMSE=64.24% | R²=0.495
Fold 3: MAE=91,298.2

## Prediksi + clustering (K-prototypes) (K-fold crossval)

In [ ]:
FILE_CLUSTERED = r"D:/DATA SKRIPSI/kontrak_sewa_bersih_clustered.xlsx"
INFLATION_FILE = r"D:/DATA SKRIPSI/Laju Inflasi Di Kota Surabaya Tahun 2006-2020.csv"
PDRB_FILE = r"D:/DATA SKRIPSI/Laju Pertumbuhan Produk Domestik Regional Bruto Atas Dasar Harga Konstan 2010 Menurut Lapangan Usaha di Kota Surabaya (persen), 2024.csv"

CLUSTER_COL = "cluster"

SHEET_CONFIGS = [
    {
        "freq_name": "Monthly",
        "SHEET_CLUSTER": "Monthly_Fixed_clustered",
        "PRICE_COL": "CuryUnitPrice",
        "sheet_name_out": "Monthly",
    },
    {
        "freq_name": "Daily",
        "SHEET_CLUSTER": "Daily_Fixed_clustered",
        "PRICE_COL": "CuryUnitPrice",
        "sheet_name_out": "Daily",
    },
]


CATEGORICAL_COLS = [
    "BusinessType", "LeaseMonthStart", "LeaseDayStart",
    "LeaseMonthEnd", "LeaseDayEnd", "TranCode",
    "ContractPeriod", "ContractType", "Building",
    "UnitArea", "UnitFloor"
]

NUMERIC_COLS = [
    "BuildingArea", "LeaseDurationDays", "LeaseDurationMonths",
    "LeaseYearStart", "LeaseYearEnd", "n_subunit",
]


PLOT_SHAP_SUMMARY = False



In [ ]:
import numpy as np
import pandas as pd
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


# Konfigurasi umum

FILE_CLUSTERED = "D:/DATA SKRIPSI/kontrak_sewa_bersih_clustered.xlsx"

SHEET_CLUSTER_MONTHLY = "Monthly_Fixed_clustered"
SHEET_CLUSTER_DAILY   = "Daily_Fixed_clustered"

CLUSTER_COL = "cluster"          
PRICE_COL   = "CuryUnitPrice"    

CATEGORICAL_COLS = [
    "BusinessType","LeaseMonthStart","LeaseDayStart",
    "LeaseMonthEnd","LeaseDayEnd","TranCode",
    "ContractPeriod","ContractType","Building", "UnitArea", "UnitFloor"
]

NUMERIC_COLS = [
    "BuildingArea","LeaseDurationDays","LeaseDurationMonths",
    "LeaseYearStart","LeaseYearEnd","n_subunit"
]


# Fungsi bantu

def clean_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = df.columns.astype(str).str.strip()
    return df

def run_catboost_for_sheet(
    file_path: str,
    sheet_cluster: str,
    out_sheet_name: str,
    writer: pd.ExcelWriter
):
    """
    Jalankan pipeline CatBoost + 5-fold CV untuk satu sheet *_Fixed_clustered,
    lalu simpan hasil OOF (cluster, harga aktual, prediksi) ke sheet Excel.
    """
    print(f"\n==============================")
    print(f"Proses sheet: {sheet_cluster}")
    print(f"Output sheet: {out_sheet_name}")
    print(f"==============================\n")

    try:
        df_main = clean_columns(pd.read_excel(file_path, sheet_name=sheet_cluster))
    except ValueError:
        df_main = clean_columns(pd.read_excel(file_path, sheet_name=0))

    for col in [CLUSTER_COL, PRICE_COL]:
        if col not in df_main.columns:
            raise KeyError(
                f"Kolom '{col}' tidak ditemukan di sheet '{sheet_cluster}'. "
                "Pastikan hasil clustering & CuryUnitPrice sudah ada di sheet tersebut."
            )

    y_all = pd.to_numeric(df_main[PRICE_COL], errors="coerce").astype(float)

    
    all_cat_cols = CATEGORICAL_COLS + [CLUSTER_COL]

    cat_all = [c for c in all_cat_cols if c in df_main.columns]
    num_all = [c for c in NUMERIC_COLS if c in df_main.columns]
    feature_cols = cat_all + num_all

    X_all = df_main[feature_cols].copy()

    for c in cat_all:
        X_all[c] = X_all[c].astype(str)

    for c in num_all:
        X_all[c] = pd.to_numeric(X_all[c], errors="coerce").astype(float)

    keep_mask = ~y_all.isna()
    X = X_all.loc[keep_mask].reset_index(drop=True)
    y = y_all.loc[keep_mask].reset_index(drop=True)

    cat_feature_idx = [X.columns.get_loc(c) for c in cat_all]

    
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    mae_scores, rmse_scores, r2_scores, rpmse_scores = [], [], [], []

    # kolom prediksi
    oof_pred = np.full(len(df_main), np.nan)
    idx_map  = np.where(keep_mask)[0]

    for fold, (tr_idx, va_idx) in enumerate(kf.split(X), 1):
        X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
        y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

        train_pool = Pool(X_tr, y_tr, cat_features=cat_feature_idx)
        valid_pool = Pool(X_va, y_va, cat_features=cat_feature_idx)

        model = CatBoostRegressor(
            loss_function="RMSE",
            depth=8,
            learning_rate=0.05,
            l2_leaf_reg=3.0,
            iterations=2000,
            random_seed=42,
            od_type="Iter",
            od_wait=100,
            verbose=False
        )
        model.fit(train_pool, eval_set=valid_pool, use_best_model=True)

        pred_va = model.predict(valid_pool)
        oof_pred[idx_map[va_idx]] = pred_va

        # hitung metrik
        mae   = mean_absolute_error(y_va, pred_va)
        rmse  = np.sqrt(mean_squared_error(y_va, pred_va))
        r2    = r2_score(y_va, pred_va)
        rpmse = (rmse / np.mean(y_va)) * 100  

        mae_scores.append(mae)
        rmse_scores.append(rmse)
        r2_scores.append(r2)
        rpmse_scores.append(rpmse)

        print(f"Fold {fold}: MAE={mae:.2f} | RMSE={rmse:.2f} | RPMSE={rpmse:.2f}% | R²={r2:.3f}")

    print("\n=== Hasil 5-Fold Cross-Validation (", out_sheet_name, ") ===")
    print(f"MAE   : {np.mean(mae_scores):.2f} ± {np.std(mae_scores):.2f}")
    print(f"RMSE  : {np.mean(rmse_scores):.2f} ± {np.std(rmse_scores):.2f}")
    print(f"RPMSE : {np.mean(rpmse_scores):.2f}% ± {np.std(rpmse_scores):.2f}%")
    print(f"R²    : {np.mean(r2_scores):.3f} ± {np.std(r2_scores):.3f}")

    
    df_out = df_main.copy()
    df_out["oof_pred"] = oof_pred

    cols_to_save = [CLUSTER_COL, PRICE_COL, "oof_pred"]
    df_out[cols_to_save].to_excel(writer, sheet_name=out_sheet_name, index=False)

    print(f"\nPrediksi OOF untuk '{out_sheet_name}' tersimpan di file Excel (sheet: {out_sheet_name})")




OUTPUT_FILE = "catboost_oof_pred_with_cluster_kfold.xlsx"

with pd.ExcelWriter(OUTPUT_FILE) as writer:
    # Monthly
    run_catboost_for_sheet(
        file_path=FILE_CLUSTERED,
        sheet_cluster=SHEET_CLUSTER_MONTHLY,
        out_sheet_name="Monthly_OOF",
        writer=writer
    )

    # Daily
    run_catboost_for_sheet(
        file_path=FILE_CLUSTERED,
        sheet_cluster=SHEET_CLUSTER_DAILY,
        out_sheet_name="Daily_OOF",
        writer=writer
    )

print(f"\nSemua hasil OOF tersimpan di file: {OUTPUT_FILE}")



Proses sheet: Monthly_Fixed_clustered
Output sheet: Monthly_OOF

Fold 1: MAE=30328.69 | RMSE=47588.57 | RPMSE=20.05% | R²=0.957
Fold 2: MAE=37558.73 | RMSE=66962.05 | RPMSE=27.07% | R²=0.927
Fold 3: MAE=32963.66 | RMSE=52074.25 | RPMSE=20.97% | R²=0.962
Fold 4: MAE=37630.79 | RMSE=64469.69 | RPMSE=26.48% | R²=0.951
Fold 5: MAE=32927.63 | RMSE=51777.19 | RPMSE=21.28% | R²=0.948

=== Hasil 5-Fold Cross-Validation ( Monthly_OOF ) ===
MAE   : 34281.90 ± 2868.88
RMSE  : 56574.35 ± 7671.36
RPMSE : 23.17% ± 2.98%
R²    : 0.949 ± 0.012

Prediksi OOF untuk 'Monthly_OOF' tersimpan di file Excel (sheet: Monthly_OOF)

Proses sheet: Daily_Fixed_clustered
Output sheet: Daily_OOF

Fold 1: MAE=2963.75 | RMSE=3971.00 | RPMSE=11.98% | R²=0.966
Fold 2: MAE=3221.35 | RMSE=4377.32 | RPMSE=12.16% | R²=0.964
Fold 3: MAE=2986.94 | RMSE=4136.26 | RPMSE=12.21% | R²=0.962
Fold 4: MAE=3095.76 | RMSE=4356.46 | RPMSE=12.64% | R²=0.964
Fold 5: MAE=3032.07 | RMSE=4022.85 | RPMSE=11.99% | R²=0.961

=== Hasil 5-Fold C

In [ ]:
df_inf = pd.read_excel("inflasi.xlsx")

df_inf_long = df_inf.melt(
    id_vars=["Bulan"],
    var_name="Year",
    value_name="Inflasi"
)

df_inf_long["Year"] = df_inf_long["Year"].astype(int)
df_inf_long["Month"] = df_inf_long["Bulan"].map({
    "Januari":1,"Februari":2,"Maret":3,"April":4,"Mei":5,"Juni":6,
    "Juli":7,"Agustus":8,"September":9,"Oktober":10,"November":11,"Desember":12
})


## Prediksi + cluster + ext. data with K-fold

In [ ]:
import os
import numpy as np
import pandas as pd

from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import shap  


# Konfigurasi file & kolom

FILE_CLUSTERED = r"D:/DATA SKRIPSI/kontrak_sewa_bersih_clustered.xlsx"
INFLATION_FILE = r"D:/DATA SKRIPSI/Laju Inflasi Di Kota Surabaya Tahun 2006-2020.csv"
PDRB_FILE = r"D:/DATA SKRIPSI/Laju Pertumbuhan Produk Domestik Regional Bruto Atas Dasar Harga Konstan 2010 Menurut Lapangan Usaha di Kota Surabaya (persen), 2024.csv"

CLUSTER_COL = "cluster"

SHEET_CONFIGS = [
    {
        "freq_name": "Monthly",
        "SHEET_CLUSTER": "Monthly_Fixed_clustered",
        "PRICE_COL": "CuryUnitPrice",
        "sheet_name_out": "Monthly_OOF",
    },
    {
        "freq_name": "Daily",
        "SHEET_CLUSTER": "Daily_Fixed_clustered",
        "PRICE_COL": "CuryUnitPrice",
        "sheet_name_out": "Daily_OOF",
    },
]

CATEGORICAL_COLS = [
    "BusinessType", "LeaseMonthStart", "LeaseDayStart",
    "LeaseMonthEnd", "LeaseDayEnd", "TranCode",
    "ContractPeriod", "ContractType", "Building",
    "UnitArea", "UnitFloor"
]

NUMERIC_COLS = [
    "BuildingArea", "LeaseDurationDays", "LeaseDurationMonths",
    "LeaseYearStart", "LeaseYearEnd", "n_subunit",
]

PLOT_SHAP_SUMMARY = False

In [ ]:

def clean_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = df.columns.astype(str).str.strip()
    return df


def parse_price_column(df: pd.DataFrame, col: str) -> pd.Series:
    s = df[col].astype(str).str.strip()
    s = s.str.replace(",", ".", regex=False)
    return pd.to_numeric(s, errors="coerce")


def load_monthly_inflation(path: str) -> pd.DataFrame:
    df_raw = pd.read_csv(path, sep=';', engine='python')
    df_raw = clean_columns(df_raw)

    year_labels = df_raw.iloc[0, 1:]

    month_map = {
        "Januari": 1, "Februari": 2, "Maret": 3, "April": 4, "Mei": 5,
        "Juni": 6, "Juli": 7, "Agustus": 8, "September": 9, "Oktober": 10,
        "November": 11, "Desember": 12
    }

    rows = []
    for i in range(3, len(df_raw)):
        month_name = df_raw.iloc[i, 0]
        if not isinstance(month_name, str):
            continue
        month_name = month_name.strip()
        if month_name == "" or "sumber" in month_name.lower():
            continue

        m = month_map.get(month_name)
        if m is None:
            continue

        for col in df_raw.columns[1:]:
            year_val = year_labels[col]
            try:
                y = int(str(year_val))
            except Exception:
                continue

            val = df_raw.at[i, col]
            if isinstance(val, str):
                val = val.replace(",", ".")
            try:
                infl_val = float(val)
            except Exception:
                continue

            rows.append({
                "LeaseYearStart": y,
                "LeaseMonthStart": m,
                "Inflation": infl_val
            })

    infl_long = pd.DataFrame(rows)
    infl_long = (
        infl_long
        .groupby(["LeaseYearStart", "LeaseMonthStart"], as_index=False)["Inflation"]
        .mean()
    )

    return infl_long



# Load Pertumbuhan PDRB

def load_pdrb_growth(path: str) -> pd.DataFrame:
    df_raw = pd.read_csv(
        path,
        sep=';',
        header=None,
        encoding='utf-8-sig'
    )

    col_year_raw = df_raw.iloc[:, 0].astype(str).str.strip()
    col_year = pd.to_numeric(col_year_raw, errors="coerce")

    col_growth_raw = df_raw.iloc[:, 1].astype(str).str.strip()
    col_growth_clean = (
        col_growth_raw
        .str.replace("%", "", regex=False)
        .str.replace(" ", "", regex=False)
        .str.replace(",", ".", regex=False)
    )
    col_growth = pd.to_numeric(col_growth_clean, errors="coerce")

    mask = ~col_year.isna() & ~col_growth.isna()

    pdrb = pd.DataFrame({
        "LeaseYearStart": col_year[mask].astype("Int64"),
        "GDP_Growth": col_growth[mask]
    }).reset_index(drop=True)

    return pdrb



# Fungsi utama per sheet (KFold + inflasi & PDRB + SHAP)
def run_catboost_for_sheet(
    file_path: str,
    sheet_cluster: str,
    out_sheet_name: str,
    writer: pd.ExcelWriter,
    price_col_name: str,
    infl_long: pd.DataFrame,
    pdrb_growth: pd.DataFrame,
):
    print(f"\n==============================")
    print(f"Proses sheet: {sheet_cluster}")
    print(f"Output sheet: {out_sheet_name}")
    print(f"==============================\n")

    try:
        df_main = clean_columns(pd.read_excel(file_path, sheet_name=sheet_cluster))
    except ValueError:
        df_main = clean_columns(pd.read_excel(file_path, sheet_name=0))

    df_main["__orig_idx__"] = np.arange(len(df_main))

    if CLUSTER_COL not in df_main.columns:
        raise KeyError(
            f"Kolom '{CLUSTER_COL}' tidak ditemukan di sheet '{sheet_cluster}'. "
            "Pastikan hasil clustering sudah ada di sheet tersebut."
        )

    if price_col_name not in df_main.columns:
        cands = [c for c in df_main.columns if "price" in c.lower()]
        if not cands:
            raise KeyError(
                f"Kolom target '{price_col_name}' tidak ditemukan dan tidak ada kolom mirip 'price' di sheet '{sheet_cluster}'."
            )
        print(f"[INFO] Kolom price '{price_col_name}' tidak ditemukan, pakai kolom mirip: {cands[0]}")
        price_col = cands[0]
    else:
        price_col = price_col_name

    # Merge Inflasi
    if {"LeaseYearStart", "LeaseMonthStart"}.issubset(df_main.columns):
        df_main = df_main.merge(
            infl_long,
            on=["LeaseYearStart", "LeaseMonthStart"],
            how="left"
        )
    else:
        print("PERINGATAN: LeaseYearStart/LeaseMonthStart tidak lengkap, Inflasi tidak bisa di-merge.")

    # Merge PDRB
    if "LeaseYearStart" in df_main.columns and "LeaseYearStart" in pdrb_growth.columns:
        df_main = df_main.merge(
            pdrb_growth,
            on="LeaseYearStart",
            how="left"
        )
    else:
        print("PERINGATAN: 'LeaseYearStart' tidak ada di pdrb_growth – GDP_Growth tidak dipakai.")

    # Target
    y_all = parse_price_column(df_main, price_col)

    # Fitur
    all_cat_cols = CATEGORICAL_COLS + [CLUSTER_COL]

    cat_all = [c for c in all_cat_cols if c in df_main.columns]
    num_all = [c for c in NUMERIC_COLS if c in df_main.columns]

    for extra in ["Inflation", "GDP_Growth"]:
        if extra in df_main.columns and extra not in num_all:
            num_all.append(extra)

    feature_cols = cat_all + num_all
    X_all = df_main[feature_cols].copy()

    for c in cat_all:
        X_all[c] = X_all[c].astype(str)
    for c in num_all:
        X_all[c] = pd.to_numeric(X_all[c], errors="coerce").astype(float)

    keep_mask = ~y_all.isna()
    X = X_all.loc[keep_mask].reset_index(drop=True)
    y = y_all.loc[keep_mask].reset_index(drop=True)

    cat_feature_idx = [X.columns.get_loc(c) for c in cat_all]

    # 5-Fold CV + SHAP
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    mae_scores, rmse_scores, r2_scores, rpmse_scores = [], [], [], []

    oof_pred = np.full(len(df_main), np.nan)
    idx_map = np.where(keep_mask)[0]

    n_rows = len(df_main)
    n_features = X.shape[1]
    oof_shap = np.full((n_rows, n_features + 1), np.nan, dtype=float)  # +1 utk base value

    for fold, (tr_idx, va_idx) in enumerate(kf.split(X), 1):
        X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
        y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

        train_pool = Pool(X_tr, y_tr, cat_features=cat_feature_idx)
        valid_pool = Pool(X_va, y_va, cat_features=cat_feature_idx)

        model = CatBoostRegressor(
            loss_function="RMSE",
            depth=8,
            learning_rate=0.05,
            l2_leaf_reg=3.0,
            iterations=2000,
            random_seed=42,
            od_type="Iter",
            od_wait=100,
            verbose=False
        )
        model.fit(train_pool, eval_set=valid_pool, use_best_model=True)

        pred_va = model.predict(valid_pool)
        oof_pred[idx_map[va_idx]] = pred_va

        shap_vals = model.get_feature_importance(
            valid_pool,
            type="ShapValues"
        ) 

        oof_shap[idx_map[va_idx], :] = shap_vals

        mae = mean_absolute_error(y_va, pred_va)
        rmse = np.sqrt(mean_squared_error(y_va, pred_va))
        r2 = r2_score(y_va, pred_va)
        rpmse = (rmse / np.mean(y_va)) * 100

        mae_scores.append(mae)
        rmse_scores.append(rmse)
        r2_scores.append(r2)
        rpmse_scores.append(rpmse)

        print(f"Fold {fold}: MAE={mae:.2f} | RMSE={rmse:.2f} | RPMSE={rpmse:.2f}% | R²={r2:.3f}")

        if PLOT_SHAP_SUMMARY and fold == 1:
            shap.summary_plot(
                shap_vals[:, :-1],
                X_va,
                feature_names=feature_cols,
                show=True
            )

    print("\n=== Hasil 5-Fold Cross-Validation (", out_sheet_name, ") ===")
    print(f"MAE   : {np.mean(mae_scores):.2f} ± {np.std(mae_scores):.2f}")
    print(f"RMSE  : {np.mean(rmse_scores):.2f} ± {np.std(rmse_scores):.2f}")
    print(f"RPMSE : {np.mean(rpmse_scores):.2f}% ± {np.std(rpmse_scores):.2f}%")
    print(f"R²    : {np.mean(r2_scores):.3f} ± {np.std(r2_scores):.3f}")

    # Simpan hasil prediksi
    df_out = df_main.copy()
    df_out["oof_pred"] = oof_pred

    cols_to_save = [
        c for c in [
            CLUSTER_COL,
            price_col,
            "Inflation",
            "GDP_Growth",
            "oof_pred",
            "__orig_idx__",
        ]
        if c in df_out.columns
    ]

    df_save = (
        df_out[cols_to_save]
        .sort_values("__orig_idx__")
        .drop(columns="__orig_idx__", errors="ignore")
    )

    df_save.to_excel(writer, sheet_name=out_sheet_name, index=False)
    print(f"\nPrediksi OOF untuk '{out_sheet_name}' tersimpan di file Excel (sheet: {out_sheet_name})")

    shap_valid = oof_shap[keep_mask, :-1]  

    # Rata-rata absolut per kolom fitur
    mean_abs_shap = np.nanmean(np.abs(shap_valid), axis=0)

    df_shap_summary = pd.DataFrame({
        "feature": feature_cols,
        "mean_abs_shap": mean_abs_shap
    }).sort_values("mean_abs_shap", ascending=False)

    shap_sheet_name = f"{out_sheet_name}_SHAP"
    df_shap_summary.to_excel(writer, sheet_name=shap_sheet_name, index=False)

    print(f"SHAP summary untuk '{out_sheet_name}' tersimpan di sheet: {shap_sheet_name}")


if __name__ == "__main__":
    infl_long = load_monthly_inflation(INFLATION_FILE)
    pdrb_growth = load_pdrb_growth(PDRB_FILE)

    OUTPUT_FILE = "catboost_oof_pred_with_cluster_kfold_with_macro_shap.xlsx"

    with pd.ExcelWriter(OUTPUT_FILE) as writer:
        for cfg in SHEET_CONFIGS:
            run_catboost_for_sheet(
                file_path=FILE_CLUSTERED,
                sheet_cluster=cfg["SHEET_CLUSTER"],
                out_sheet_name=cfg["sheet_name_out"],
                writer=writer,
                price_col_name=cfg["PRICE_COL"],
                infl_long=infl_long,
                pdrb_growth=pdrb_growth,
            )

    print(f"\nSemua hasil OOF dan SHAP summary tersimpan di file: {OUTPUT_FILE}")


C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Proses sheet: Monthly_Fixed_clustered
Output sheet: Monthly_OOF

Fold 1: MAE=30263.20 | RMSE=46022.97 | RPMSE=19.39% | R²=0.959
Fold 2: MAE=37542.03 | RMSE=63500.73 | RPMSE=25.67% | R²=0.935
Fold 3: MAE=32575.87 | RMSE=51055.47 | RPMSE=20.56% | R²=0.963
Fold 4: MAE=37619.42 | RMSE=64458.63 | RPMSE=26.48% | R²=0.951
Fold 5: MAE=33228.57 | RMSE=51378.49 | RPMSE=21.12% | R²=0.948

=== Hasil 5-Fold Cross-Validation ( Monthly_OOF ) ===
MAE   : 34245.82 ± 2895.89
RMSE  : 55283.26 ± 7356.48
RPMSE : 22.64% ± 2.87%
R²    : 0.951 ± 0.010

Prediksi OOF untuk 'Monthly_OOF' tersimpan di file Excel (sheet: Monthly_OOF)
SHAP summary untuk 'Monthly_OOF' tersimpan di sheet: Monthly_OOF_SHAP

Proses sheet: Daily_Fixed_clustered
Output sheet: Daily_OOF

Fold 1: MAE=2949.07 | RMSE=3973.72 | RPMSE=11.99% | R²=0.966
Fold 2: MAE=3168.78 | RMSE=4293.10 | RPMSE=11.93% | R²=0.966
Fold 3: MAE=2988.85 | RMSE=4144.01 | RPMSE=12.24% | R²=0.962
Fold 4: MAE=3171.60 | RMSE=4349.95 | RPMSE=12.63% | R²=0.964
Fold 5: MA

## Prediksi + cluster with walk forward CV

In [ ]:
import numpy as np
import pandas as pd
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import TimeSeriesSplit   
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
# Konfigurasi umum
FILE_CLUSTERED = "D:/DATA SKRIPSI/kontrak_sewa_bersih_clustered.xlsx"
CLUSTER_COL    = "cluster"   

CATEGORICAL_COLS = [
    "BusinessType","LeaseMonthStart","LeaseDayStart",
    "LeaseMonthEnd","LeaseDayEnd","TranCode",
    "ContractPeriod","ContractType","Building", "UnitArea", "UnitFloor"
    
]

NUMERIC_COLS = [
    "BuildingArea","LeaseDurationDays","LeaseDurationMonths",
    "LeaseYearStart","LeaseYearEnd","n_subunit"
]

PRICE_COL_NAME = "CuryUnitPrice"      


def clean_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = df.columns.astype(str).str.strip()
    return df

def run_catboost_for_period(
    file_path: str,
    sheet_cluster: str,
    sheet_price: str,
    price_col: str = PRICE_COL_NAME,
    cluster_col: str = CLUSTER_COL,
    output_prefix: str = "catboost_oof_pred_with_cluster_walkforward"
):
    print(f"\n========== PERIODE: {sheet_cluster.replace('_clustered','')} ==========")

    
    # Load sheet utama (+ cluster)
    
    try:
        df_main = clean_columns(pd.read_excel(file_path, sheet_name=sheet_cluster))
    except ValueError:
        df_main = clean_columns(pd.read_excel(file_path, sheet_name=0))

    if cluster_col not in df_main.columns:
        raise KeyError(
            f"Kolom cluster '{cluster_col}' tidak ditemukan di sheet '{sheet_cluster}'. "
            "Pastikan hasil clustering sudah disimpan sebagai kolom tersebut."
        )

    xls = pd.ExcelFile(file_path)
    if sheet_price not in xls.sheet_names:
        raise KeyError(f"Sheet '{sheet_price}' tidak ditemukan di {file_path}.")

    df_price = clean_columns(pd.read_excel(file_path, sheet_name=sheet_price))

    target_price_col = price_col
    if target_price_col not in df_price.columns:
        cands = [c for c in df_price.columns if "price" in c.lower()]
        if not cands:
            raise KeyError(
                f"Kolom target '{price_col}' tidak ditemukan dan tidak ada kolom mirip 'price' di sheet '{sheet_price}'."
            )
        target_price_col = cands[0]
        print(f"Kolom harga default '{price_col}' tidak ditemukan, pakai kolom '{target_price_col}'.")

    # mapping harga ke baris df_main
    if "RowID" in df_price.columns:
        df_price["RowID"] = pd.to_numeric(df_price["RowID"], errors="coerce").astype("Int64")
        s_price = df_price.set_index("RowID")[target_price_col]
        y_all = pd.Series(df_main.index, index=df_main.index).map(s_price).astype(float)
    else:
        if len(df_price) != len(df_main):
            raise ValueError(
                f"Panjang sheet utama ({len(df_main)}) berbeda dengan sheet '{sheet_price}' ({len(df_price)}). "
                "Tambahkan kolom RowID di price sheet agar bisa dipetakan."
            )
        y_all = (
            pd.to_numeric(df_price[target_price_col], errors="coerce")
            .astype(float)
            .reset_index(drop=True)
        )
    
    need_cols = {"LeaseYearStart", "LeaseMonthStart", "LeaseDayStart"}
    if not need_cols.issubset(df_main.columns):
        raise KeyError(
            "Untuk walk-forward CV dibutuhkan kolom LeaseYearStart, LeaseMonthStart, LeaseDayStart."
        )

    lease_start_date = pd.to_datetime(
        dict(
            year=df_main["LeaseYearStart"],
            month=df_main["LeaseMonthStart"],
            day=df_main["LeaseDayStart"],
        ),
        errors="coerce",
    )

    # buang baris yang tidak punya tanggal valid
    valid_date_mask = ~lease_start_date.isna()
    df_main = df_main.loc[valid_date_mask].reset_index(drop=True)
    y_all   = y_all.loc[valid_date_mask].reset_index(drop=True)
    lease_start_date = lease_start_date.loc[valid_date_mask].reset_index(drop=True)

    # sort berdasarkankolom LeaseStartDate
    order = np.argsort(lease_start_date.values)
    df_main = df_main.iloc[order].reset_index(drop=True)
    y_all   = y_all.iloc[order].reset_index(drop=True)
    lease_start_date = lease_start_date.iloc[order].reset_index(drop=True)

    #siapkan kolom input
    all_cat_cols = CATEGORICAL_COLS + [cluster_col]

    cat_all = [c for c in all_cat_cols if c in df_main.columns]
    num_all = [c for c in NUMERIC_COLS if c in df_main.columns]
    feature_cols = cat_all + num_all

    X_all = df_main[feature_cols].copy()

    for c in cat_all:
        X_all[c] = X_all[c].astype(str)

    for c in num_all:
        X_all[c] = pd.to_numeric(X_all[c], errors="coerce").astype(float)

    keep_mask = ~y_all.isna()
    X = X_all.loc[keep_mask].reset_index(drop=True)
    y = y_all.loc[keep_mask].reset_index(drop=True)

    cat_feature_idx = [X.columns.get_loc(c) for c in cat_all]

    
    # Walk-Forward Cross Validation dengan CatBoost (TimeSeriesSplit)
    
    tscv = TimeSeriesSplit(n_splits=5)

    mae_scores, rmse_scores, r2_scores, rpmse_scores = [], [], [], []

    # prediksi
    oof_pred = np.full(len(df_main), np.nan)
    idx_map  = np.where(keep_mask)[0]   

    for fold, (tr_idx, va_idx) in enumerate(tscv.split(X), 1):
        X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
        y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

        train_pool = Pool(X_tr, y_tr, cat_features=cat_feature_idx)
        valid_pool = Pool(X_va, y_va, cat_features=cat_feature_idx)

        model = CatBoostRegressor(
            loss_function="RMSE",
            depth=8,
            learning_rate=0.05,
            l2_leaf_reg=3.0,
            iterations=2000,
            random_seed=42,
            od_type="Iter",
            od_wait=100,
            verbose=False
        )
        model.fit(train_pool, eval_set=valid_pool, use_best_model=True)

        pred_va = model.predict(valid_pool)

        oof_pred[idx_map[va_idx]] = pred_va

        # hitung metrik
        mae   = mean_absolute_error(y_va, pred_va)
        rmse  = np.sqrt(mean_squared_error(y_va, pred_va))
        r2    = r2_score(y_va, pred_va)
        rpmse = (rmse / np.mean(y_va)) * 100  

        mae_scores.append(mae)
        rmse_scores.append(rmse)
        r2_scores.append(r2)
        rpmse_scores.append(rpmse)

        print(f"Fold {fold}: MAE={mae:.2f} | RMSE={rmse:.2f} | RPMSE={rpmse:.2f}% | R²={r2:.3f}")

    print("\n=== Hasil Walk-Forward CV (TimeSeriesSplit) ===")
    print(f"MAE   : {np.mean(mae_scores):.2f} ± {np.std(mae_scores):.2f}")
    print(f"RMSE  : {np.mean(rmse_scores):.2f} ± {np.std(rmse_scores):.2f}")
    print(f"RPMSE : {np.mean(rpmse_scores):.2f}% ± {np.std(rpmse_scores):.2f}%")
    print(f"R²    : {np.mean(r2_scores):.3f} ± {np.std(r2_scores):.3f}")

    
    # Simpan hasil
    
    df_out = df_main.copy()
    df_out["oof_pred"] = oof_pred

    if target_price_col not in df_out.columns:
        df_out[target_price_col] = y_all.values

    cols_to_save = [cluster_col, target_price_col, "oof_pred"]

    period_label = sheet_cluster.replace("_clustered", "")
    out_file = f"{output_prefix}_{period_label}.xlsx"
    df_out[cols_to_save].to_excel(out_file, index=False)

    print(f"\nPrediksi OOF tersimpan di: {out_file}")



run_catboost_for_period(
    file_path=FILE_CLUSTERED,
    sheet_cluster="Monthly_Fixed_clustered",
    sheet_price="Monthly_Fixed_CuryUnitPrice",
    output_prefix="catboost_oof_pred_with_cluster_walkforward"
)

run_catboost_for_period(
    file_path=FILE_CLUSTERED,
    sheet_cluster="Daily_Fixed_clustered",
    sheet_price="Daily_Fixed_CuryUnitPrice",
    output_prefix="catboost_oof_pred_with_cluster_walkforward"
)



========== PERIODE: Monthly_Fixed ==========
Fold 1: MAE=50525.23 | RMSE=76711.82 | RPMSE=30.36% | R²=0.886
Fold 2: MAE=42265.84 | RMSE=70642.76 | RPMSE=30.23% | R²=0.889
Fold 3: MAE=37245.78 | RMSE=57333.29 | RPMSE=24.25% | R²=0.936
Fold 4: MAE=43071.62 | RMSE=60936.85 | RPMSE=25.50% | R²=0.920
Fold 5: MAE=72020.08 | RMSE=227851.16 | RPMSE=94.70% | R²=0.633

=== Hasil Walk-Forward CV (TimeSeriesSplit) ===
MAE   : 49025.71 ± 12254.82
RMSE  : 98695.18 ± 64943.01
RPMSE : 41.01% ± 26.96%
R²    : 0.853 ± 0.111

Prediksi OOF tersimpan di: catboost_oof_pred_with_cluster_walkforward_Monthly_Fixed.xlsx

========== PERIODE: Daily_Fixed ==========
Fold 1: MAE=4244.41 | RMSE=5594.90 | RPMSE=16.80% | R²=0.925
Fold 2: MAE=3994.12 | RMSE=5043.17 | RPMSE=13.28% | R²=0.961
Fold 3: MAE=4039.10 | RMSE=5360.00 | RPMSE=14.40% | R²=0.951
Fold 4: MAE=4296.00 | RMSE=5452.65 | RPMSE=17.03% | R²=0.932
Fold 5: MAE=3648.21 | RMSE=4796.38 | RPMSE=14.88% | R²=0.934

=== Hasil Walk-Forward CV (TimeSeriesSplit) ===

## Prediksi + cluster + Inflation data (walkforward)

In [ ]:
FILE_CLUSTERED = "D:/DATA SKRIPSI/kontrak_sewa_bersih_clustered.xlsx"
INFLATION_FILE = "D:/DATA SKRIPSI/Laju Inflasi Di Kota Surabaya Tahun 2006-2020.csv"
CLUSTER_COL    = "cluster"  

SHEET_CONFIGS = [
    {
        "freq_name": "Monthly",
        "SHEET_CLUSTER": "Monthly_Fixed_clustered",
        "PRICE_SHEET": "Monthly_Fixed_CuryUnitPrice",
        "PRICE_COL": "CuryUnitPrice",
        "OUTPUT_FILE": "catboost_oof_pred_with_cluster_walkforward_Monthly.xlsx",
    },
    {
        "freq_name": "Daily",
        "SHEET_CLUSTER": "Daily_Fixed_clustered",
        "PRICE_SHEET": "Daily_Fixed_CuryUnitPrice",
        "PRICE_COL": "CuryUnitPrice",
        "OUTPUT_FILE": "catboost_oof_pred_with_cluster_walkforward_Daily.xlsx",
    },
]

CATEGORICAL_COLS = [
    "BusinessType","LeaseMonthStart","LeaseDayStart",
    "LeaseMonthEnd","LeaseDayEnd","TranCode",
    "ContractPeriod","ContractType","Building", "UnitArea", "UnitFloor"
]

NUMERIC_COLS = [
    "BuildingArea","LeaseDurationDays","LeaseDurationMonths",
    "LeaseYearStart","LeaseYearEnd","n_subunit"
]

In [ ]:
def clean_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = df.columns.astype(str).str.strip()
    return df

def parse_price_column(df: pd.DataFrame, col: str) -> pd.Series:
    
    s = df[col].astype(str).str.strip()
    s = s.str.replace(",", ".", regex=False)   
    return pd.to_numeric(s, errors="coerce")


def load_monthly_inflation(path: str) -> pd.DataFrame:
    """
    Membaca file inflasi BPS (format wide: kolom tahun 2015, 2016, ...; baris bulan)
    lalu mengembalikan DataFrame long: Year, Month, Inflation
    """
    df_raw = pd.read_csv(path, sep=';', engine='python')
    df_raw = clean_columns(df_raw)

    # nama kolom pertama = nama bulan
    month_col = df_raw.columns[0]

    # baris pertama (index 0) berisi label tahun untuk kolom-kolom lain
    year_labels = df_raw.iloc[0, 1:]  # kolom 1 dst

    month_map = {
        "Januari": 1, "Februari": 2, "Maret": 3, "April": 4, "Mei": 5,
        "Juni": 6, "Juli": 7, "Agustus": 8, "September": 9, "Oktober": 10,
        "November": 11, "Desember": 12
    }

    rows = []
    # data bulan mulai di baris index 3 (lihat file asli)
    for i in range(3, len(df_raw)):
        month_name = df_raw.iloc[i, 0]
        if not isinstance(month_name, str):
            continue
        month_name = month_name.strip()
        if month_name == "" or "sumber" in month_name.lower():
            continue

        m = month_map.get(month_name)
        if m is None:
            continue

        # loop setiap kolom tahun
        for col in df_raw.columns[1:]:
            year_val = year_labels[col]
            try:
                y = int(str(year_val))
            except Exception:
                continue

            val = df_raw.at[i, col]
            if isinstance(val, str):
                val = val.replace(",", ".")
            try:
                infl_val = float(val)
            except Exception:
                continue

            rows.append({"LeaseYearStart": y, "LeaseMonthStart": m, "Inflation": infl_val})

    infl_long = pd.DataFrame(rows)

    # jika ada duplikat (Year, Month) ambil rata-rata
    infl_long = (
        infl_long
        .groupby(["LeaseYearStart", "LeaseMonthStart"], as_index=False)["Inflation"]
        .mean()
    )

    return infl_long



# Fungsi utama untuk memproses satu pasangan sheet (Monthly / Daily)
def run_for_sheet(
    xls: pd.ExcelFile,
    sheet_cluster: str,
    price_sheet: str,
    price_col_name: str,
    output_file: str,
):
    print(f"\n================= Proses {sheet_cluster} / {price_sheet} =================")

    # --------------------------------------------------
    # Load sheet utama (sudah termasuk kolom cluster)
    # --------------------------------------------------
    if sheet_cluster not in xls.sheet_names:
        raise KeyError(
            f"Sheet cluster '{sheet_cluster}' tidak ditemukan di {FILE_CLUSTERED}."
        )

    try:
        df_main = clean_columns(pd.read_excel(xls, sheet_name=sheet_cluster))
    except ValueError:
        df_main = clean_columns(pd.read_excel(xls, sheet_name=0))

    # pastikan kolom cluster ada
    if CLUSTER_COL not in df_main.columns:
        raise KeyError(
            f"Kolom cluster '{CLUSTER_COL}' tidak ditemukan di sheet '{sheet_cluster}'. "
            "Pastikan hasil clustering sudah disimpan sebagai kolom tersebut."
        )

    # --------------------------------------------------
    # Load price sheet dan selaraskan dengan data utama
    # --------------------------------------------------
    if price_sheet not in xls.sheet_names:
        raise KeyError(f"Sheet '{price_sheet}' tidak ditemukan di {FILE_CLUSTERED}.")

    df_price = clean_columns(pd.read_excel(xls, sheet_name=price_sheet))

    # gunakan local variable untuk kolom price
    price_col = price_col_name

    # pastikan kolom harga ada (atau fallback cari yang mengandung 'price')
    if price_col not in df_price.columns:
        cands = [c for c in df_price.columns if "price" in c.lower()]
        if not cands:
            raise KeyError(
                f"Kolom target '{price_col}' tidak ditemukan dan tidak ada kolom mirip 'price' di sheet '{price_sheet}'."
            )
        price_col = cands[0]

    # mapping harga ke baris df_main
    if "RowID" in df_price.columns:
        # map berdasarkan RowID -> baris df_main (diasumsikan RowID = index asli 0..n-1)
        df_price["RowID"] = pd.to_numeric(df_price["RowID"], errors="coerce").astype("Int64")
        s_price = df_price.set_index("RowID")[price_col]
        y_all = pd.Series(df_main.index, index=df_main.index).map(s_price).astype(float)
    else:
        # kalau tidak ada RowID, sejajarkan by-position (panjang harus sama)
        if len(df_price) != len(df_main):
            raise ValueError(
                f"Panjang sheet utama ({len(df_main)}) berbeda dengan sheet '{price_sheet}' ({len(df_price)}). "
                "Tambahkan kolom RowID di price sheet agar bisa dipetakan."
            )
        y_all = (
            pd.to_numeric(df_price[price_col], errors="coerce")
            .astype(float)
            .reset_index(drop=True)
        )

    # --------------------------------------------------
    # URUTKAN BERDASARKAN TANGGAL (UNTUK WALK-FORWARD)
    # --------------------------------------------------
    need_cols = {"LeaseYearStart", "LeaseMonthStart", "LeaseDayStart"}
    if not need_cols.issubset(df_main.columns):
        raise KeyError(
            "Untuk walk-forward CV dibutuhkan kolom LeaseYearStart, LeaseMonthStart, LeaseDayStart."
        )

    lease_start_date = pd.to_datetime(
        dict(
            year=df_main["LeaseYearStart"],
            month=df_main["LeaseMonthStart"],
            day=df_main["LeaseDayStart"],
        ),
        errors="coerce",
    )

    # buang baris yang tidak punya tanggal valid
    valid_date_mask = ~lease_start_date.isna()
    df_main = df_main.loc[valid_date_mask].reset_index(drop=True)
    y_all   = y_all.loc[valid_date_mask].reset_index(drop=True)
    lease_start_date = lease_start_date.loc[valid_date_mask].reset_index(drop=True)

    # sort berdasarkan tanggal mulai sewa
    order = np.argsort(lease_start_date.values)
    df_main = df_main.iloc[order].reset_index(drop=True)
    y_all   = y_all.iloc[order].reset_index(drop=True)
    lease_start_date = lease_start_date.iloc[order].reset_index(drop=True)

    # --------------------------------------------------
    # Siapkan fitur (termasuk kolom cluster sebagai kategori)
    # --------------------------------------------------
    all_cat_cols = CATEGORICAL_COLS + [CLUSTER_COL]

    cat_all = [c for c in all_cat_cols if c in df_main.columns]
    num_all = [c for c in NUMERIC_COLS if c in df_main.columns]
    feature_cols = cat_all + num_all

    X_all = df_main[feature_cols].copy()

    # kategorikal -> string
    for c in cat_all:
        X_all[c] = X_all[c].astype(str)

    # numerik -> float
    for c in num_all:
        X_all[c] = pd.to_numeric(X_all[c], errors="coerce").astype(float)

    # buang baris tanpa target
    keep_mask = ~y_all.isna()
    X = X_all.loc[keep_mask].reset_index(drop=True)
    y = y_all.loc[keep_mask].reset_index(drop=True)

    # index kolom kategorikal untuk CatBoost
    cat_feature_idx = [X.columns.get_loc(c) for c in cat_all]

    # --------------------------------------------------
    # Walk-Forward Cross Validation dengan CatBoost
    # --------------------------------------------------
    tscv = TimeSeriesSplit(n_splits=5)

    mae_scores, rmse_scores, r2_scores, rpmse_scores = [], [], [], []

    # oof_pred untuk seluruh df_main (setelah sort & filter tanggal)
    oof_pred = np.full(len(df_main), np.nan)
    idx_map  = np.where(keep_mask)[0]   # posisi baris yang punya target, di df_main yang sudah disort

    for fold, (tr_idx, va_idx) in enumerate(tscv.split(X), 1):
        X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
        y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

        train_pool = Pool(X_tr, y_tr, cat_features=cat_feature_idx)
        valid_pool = Pool(X_va, y_va, cat_features=cat_feature_idx)

        model = CatBoostRegressor(
            loss_function="RMSE",
            depth=8,
            learning_rate=0.05,
            l2_leaf_reg=3.0,
            iterations=2000,
            random_seed=42,
            od_type="Iter",
            od_wait=100,
            verbose=False
        )
        model.fit(train_pool, eval_set=valid_pool, use_best_model=True)

        # ==================================================
        # Hitung Train RMSE
        # ==================================================
        pred_tr = model.predict(train_pool)
        rmse_train = np.sqrt(mean_squared_error(y_tr, pred_tr))

        # ==================================================
        # Prediksi Validation
        # ==================================================
        pred_va = model.predict(valid_pool)
        oof_pred[idx_map[va_idx]] = pred_va

        # ==================================================
        # Hitung Validation Metrics
        # ==================================================
        mae   = mean_absolute_error(y_va, pred_va)
        rmse  = np.sqrt(mean_squared_error(y_va, pred_va))
        r2    = r2_score(y_va, pred_va)
        rpmse = (rmse / np.mean(y_va)) * 100

        print(
            f"[{sheet_cluster}] Fold {fold}: "
            f"Train RMSE={rmse_train:.2f} | "
            f"Val RMSE={rmse:.2f} | "
            f"MAE={mae:.2f} | "
            f"RPMSE={rpmse:.2f}% | "
            f"R²={r2:.3f}"
        )

        mae_scores.append(mae)
        rmse_scores.append(rmse)
        r2_scores.append(r2)
        rpmse_scores.append(rpmse)


        # print(f"[{sheet_cluster}] Fold {fold}: MAE={mae:.2f} | RMSE={rmse:.2f} | RPMSE={rpmse:.2f}% | R²={r2:.3f}")

    print(f"\n=== Hasil Walk-Forward CV (TimeSeriesSplit) untuk {sheet_cluster} ===")
    print(f"MAE   : {np.mean(mae_scores):.2f} ± {np.std(mae_scores):.2f}")
    print(f"RMSE  : {np.mean(rmse_scores):.2f} ± {np.std(rmse_scores):.2f}")
    print(f"RPMSE : {np.mean(rpmse_scores):.2f}% ± {np.std(rpmse_scores):.2f}%")
    print(f"R²    : {np.mean(r2_scores):.3f} ± {np.std(r2_scores):.3f}")

    # --------------------------------------------------
    # Simpan hasil OOF (termasuk cluster, harga aktual, prediksi)
    # --------------------------------------------------
    df_out = df_main.copy()
    df_out["oof_pred"] = oof_pred

    # tambahkan kolom harga aktual jika belum ada
    if price_col not in df_out.columns:
        df_out[price_col] = y_all.values

    # pastikan kolom cluster ikut tersimpan
    cols_to_save = [CLUSTER_COL, price_col, "oof_pred"]
    df_out[cols_to_save].to_excel(output_file, index=False)

    print(f"\nPrediksi OOF tersimpan di: {output_file}")



In [22]:

# Main
xls = pd.ExcelFile(FILE_CLUSTERED)

for cfg in SHEET_CONFIGS:
    run_for_sheet(
        xls=xls,
        sheet_cluster=cfg["SHEET_CLUSTER"],
        price_sheet=cfg["PRICE_SHEET"],
        price_col_name=cfg["PRICE_COL"],
        output_file=cfg["OUTPUT_FILE"],
    )


================= Proses Monthly_Fixed_clustered / Monthly_Fixed_CuryUnitPrice =================
[Monthly_Fixed_clustered] Fold 1: Train RMSE=45627.04 | Val RMSE=76711.82 | MAE=50525.23 | RPMSE=30.36% | R²=0.886
[Monthly_Fixed_clustered] Fold 2: Train RMSE=38999.86 | Val RMSE=70642.76 | MAE=42265.84 | RPMSE=30.23% | R²=0.889
[Monthly_Fixed_clustered] Fold 3: Train RMSE=40146.31 | Val RMSE=57333.29 | MAE=37245.78 | RPMSE=24.25% | R²=0.936
[Monthly_Fixed_clustered] Fold 4: Train RMSE=53871.20 | Val RMSE=60936.85 | MAE=43071.62 | RPMSE=25.50% | R²=0.920
[Monthly_Fixed_clustered] Fold 5: Train RMSE=39160.72 | Val RMSE=227851.16 | MAE=72020.08 | RPMSE=94.70% | R²=0.633

=== Hasil Walk-Forward CV (TimeSeriesSplit) untuk Monthly_Fixed_clustered ===
MAE   : 49025.71 ± 12254.82
RMSE  : 98695.18 ± 64943.01
RPMSE : 41.01% ± 26.96%
R²    : 0.853 ± 0.111

Prediksi OOF tersimpan di: catboost_oof_pred_with_cluster_walkforward_Monthly.xlsx

================= Proses Daily_Fixed_clustered / Daily_Fixed

### ini + PDRB

In [118]:
FILE_CLUSTERED = r"D:/DATA SKRIPSI/kontrak_sewa_bersih_clustered.xlsx"
INFLATION_FILE = r"D:/DATA SKRIPSI/Laju Inflasi Di Kota Surabaya Tahun 2006-2020.csv"
PDRB_FILE = r"D:/DATA SKRIPSI/Laju Pertumbuhan Produk Domestik Regional Bruto Atas Dasar Harga Konstan 2010 Menurut Lapangan Usaha di Kota Surabaya (persen), 2024.csv"

CLUSTER_COL = "cluster"

SHEET_CONFIGS = [
    {
        "freq_name": "Monthly",
        "SHEET_CLUSTER": "Monthly_Fixed_clustered",
        "PRICE_COL": "CuryUnitPrice",
        "sheet_name_out": "Monthly",
    },
    {
        "freq_name": "Daily",
        "SHEET_CLUSTER": "Daily_Fixed_clustered",
        "PRICE_COL": "CuryUnitPrice",
        "sheet_name_out": "Daily",
    },
]


CATEGORICAL_COLS = [
    "BusinessType", "LeaseMonthStart", "LeaseDayStart",
    "LeaseMonthEnd", "LeaseDayEnd", "TranCode",
    "ContractPeriod", "ContractType", "Building",
    "UnitArea", "UnitFloor"
]

NUMERIC_COLS = [
    "BuildingArea", "LeaseDurationDays", "LeaseDurationMonths",
    "LeaseYearStart", "LeaseYearEnd", "n_subunit",
]

# Flag opsional: kalau mau langsung menampilkan summary plot SHAP
PLOT_SHAP_SUMMARY = False



In [ ]:
def clean_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = df.columns.astype(str).str.strip()
    return df


def parse_price_column(df: pd.DataFrame, col: str) -> pd.Series:
    s = df[col].astype(str).str.strip()
    s = s.str.replace(",", ".", regex=False)
    return pd.to_numeric(s, errors="coerce")


# ============================================
# Load Inflasi Bulanan (wide → long)
# ============================================
def load_monthly_inflation(path: str) -> pd.DataFrame:
    df_raw = pd.read_csv(path, sep=';', engine='python')
    df_raw = clean_columns(df_raw)

    # baris pertama (index 0) berisi label tahun untuk kolom-kolom lain
    year_labels = df_raw.iloc[0, 1:]

    month_map = {
        "Januari": 1, "Februari": 2, "Maret": 3, "April": 4, "Mei": 5,
        "Juni": 6, "Juli": 7, "Agustus": 8, "September": 9, "Oktober": 10,
        "November": 11, "Desember": 12
    }

    rows = []
    # data bulan mulai di baris index 3 (sesuai file inflasi)
    for i in range(3, len(df_raw)):
        month_name = df_raw.iloc[i, 0]
        if not isinstance(month_name, str):
            continue
        month_name = month_name.strip()
        if month_name == "" or "sumber" in month_name.lower():
            continue

        m = month_map.get(month_name)
        if m is None:
            continue

        for col in df_raw.columns[1:]:
            year_val = year_labels[col]
            try:
                y = int(str(year_val))
            except Exception:
                continue

            val = df_raw.at[i, col]
            if isinstance(val, str):
                val = val.replace(",", ".")
            try:
                infl_val = float(val)
            except Exception:
                continue

            rows.append({
                "LeaseYearStart": y,
                "LeaseMonthStart": m,
                "Inflation": infl_val
            })

    infl_long = pd.DataFrame(rows)

    infl_long = (
        infl_long
        .groupby(["LeaseYearStart", "LeaseMonthStart"], as_index=False)["Inflation"]
        .mean()
    )

    return infl_long


# ============================================
# Load Pertumbuhan PDRB (bentuk seperti screenshot)
# ============================================
def load_pdrb_growth(path: str) -> pd.DataFrame:
    import pandas as pd

    # Paksa pakai delimiter ';'
    df_raw = pd.read_csv(
        path,
        sep=';',            # <--- ganti ini
        header=None,
        encoding='utf-8-sig'  # buang karakter BOM di awal "﻿Lapangan..."
    )
    # kalau mau, baru panggil clean_columns di sini
    # df_raw = clean_columns(df_raw)

    # --- kolom 0: tahun ---
    col_year_raw = df_raw.iloc[:, 0].astype(str).str.strip()
    col_year = pd.to_numeric(col_year_raw, errors="coerce")

    # --- kolom 1: pertumbuhan ---
    col_growth_raw = df_raw.iloc[:, 1].astype(str).str.strip()
    col_growth_clean = (
        col_growth_raw
        .str.replace("%", "", regex=False)
        .str.replace(" ", "", regex=False)
        .str.replace(",", ".", regex=False)
    )
    col_growth = pd.to_numeric(col_growth_clean, errors="coerce")

    mask = ~col_year.isna() & ~col_growth.isna()

    pdrb = pd.DataFrame({
        "LeaseYearStart": col_year[mask].astype("Int64"),
        "GDP_Growth": col_growth[mask]
    }).reset_index(drop=True)

    return pdrb


# Fungsi utama per sheet


OOF_FILE = "catboost_oof_pred_with_cluster_walkforward.xlsx"
SHAP_FILE = "catboost_oof_pred_with_cluster_walkforward_SHAP_importance.xlsx"


def run_for_sheet(
    xls: pd.ExcelFile,
    sheet_cluster: str,
    price_col_name: str,
    sheet_name_out: str,
    infl_long: pd.DataFrame,
    pdrb_growth: pd.DataFrame,
):
    print(f"\n================= Proses {sheet_cluster} =================")

    # ----------------------------------------------------
    # BEST PARAMETER PER PERIODE
    # ----------------------------------------------------
    BEST_CB_PARAMS = {
        "monthly": {
            "depth": 4,
            "learning_rate": 0.017,
            "l2_leaf_reg": 9.951,
            "iterations": 1491,
        },
        "daily": {
            "depth": 8,
            "learning_rate": 0.05,
            "l2_leaf_reg": 3.0,
            "iterations": 2000,
        },
    }

    sheet_lower = sheet_cluster.lower()
    if "monthly" in sheet_lower:
        period_key = "monthly"
    elif "daily" in sheet_lower:
        period_key = "daily"
    else:
        period_key = None

    if period_key in BEST_CB_PARAMS:
        cb_params = BEST_CB_PARAMS[period_key]
    else:
        cb_params = {
            "depth": 8,
            "learning_rate": 0.05,
            "l2_leaf_reg": 3.0,
            "iterations": 2000,
        }

    # ===================== LOAD SHEET CLUSTER =====================
    if sheet_cluster not in xls.sheet_names:
        raise KeyError(
            f"Sheet cluster '{sheet_cluster}' tidak ditemukan di {FILE_CLUSTERED}."
        )

    try:
        df_main = clean_columns(pd.read_excel(xls, sheet_name=sheet_cluster))
    except ValueError:
        df_main = clean_columns(pd.read_excel(xls, sheet_name=0))

    # simpan urutan asli sebelum ada filtering/sorting apa pun
    df_main["__orig_idx__"] = np.arange(len(df_main))

    if CLUSTER_COL not in df_main.columns:
        raise KeyError(
            f"Kolom cluster '{CLUSTER_COL}' tidak ditemukan di sheet '{sheet_cluster}'. "
            "Pastikan hasil clustering sudah disimpan sebagai kolom tersebut."
        )

    # ---- merge Inflasi dan PDRB ----
    if {"LeaseYearStart", "LeaseMonthStart"}.issubset(df_main.columns):
        df_main = df_main.merge(
            infl_long,
            on=["LeaseYearStart", "LeaseMonthStart"],
            how="left"
        )

    if "LeaseYearStart" in df_main.columns and "LeaseYearStart" in pdrb_growth.columns:
        df_main = df_main.merge(
            pdrb_growth,
            on="LeaseYearStart",
            how="left"
        )
    else:
        print("PERINGATAN: 'LeaseYearStart' tidak ada di pdrb_growth – GDP_Growth tidak dipakai.")

    # ===================== LOAD PRICE DARI SHEET CLUSTER =====================
    price_col = price_col_name
    if price_col not in df_main.columns:
        # fallback cari kolom yang mengandung kata 'price'
        cands = [c for c in df_main.columns if "price" in c.lower()]
        if not cands:
            raise KeyError(
                f"Kolom target '{price_col}' tidak ditemukan dan tidak ada kolom mirip 'price' di sheet '{sheet_cluster}'."
            )
        price_col = cands[0]
        print(f"[INFO] Kolom price tidak ditemukan, pakai kolom mirip: {price_col}")

    # konversi harga ke numerik (pakai helper yang sudah ada)
    y_all = parse_price_column(df_main, price_col)

    # ---- urutkan berdasarkan tanggal ----
    need_cols = {"LeaseYearStart", "LeaseMonthStart", "LeaseDayStart"}
    if not need_cols.issubset(df_main.columns):
        raise KeyError(
            "Untuk walk-forward CV dibutuhkan kolom LeaseYearStart, LeaseMonthStart, LeaseDayStart."
        )

    lease_start_date = pd.to_datetime(
        dict(
            year=df_main["LeaseYearStart"],
            month=df_main["LeaseMonthStart"],
            day=df_main["LeaseDayStart"],
        ),
        errors="coerce",
    )

    valid_date_mask = ~lease_start_date.isna()
    df_main = df_main.loc[valid_date_mask].reset_index(drop=True)
    y_all = y_all.loc[valid_date_mask].reset_index(drop=True)
    lease_start_date = lease_start_date.loc[valid_date_mask].reset_index(drop=True)

    order = np.argsort(lease_start_date.values)
    df_main = df_main.iloc[order].reset_index(drop=True)
    y_all = y_all.iloc[order].reset_index(drop=True)
    lease_start_date = lease_start_date.iloc[order].reset_index(drop=True)

    # ---- fitur ----
    all_cat_cols = CATEGORICAL_COLS + [CLUSTER_COL]

    cat_all = [c for c in all_cat_cols if c in df_main.columns]
    num_all = [c for c in NUMERIC_COLS if c in df_main.columns]

    for extra in ["Inflation", "GDP_Growth"]:
        if extra in df_main.columns and extra not in num_all:
            num_all.append(extra)

    feature_cols = cat_all + num_all

    X_all = df_main[feature_cols].copy()

    for c in cat_all:
        X_all[c] = X_all[c].astype(str)
    for c in num_all:
        X_all[c] = pd.to_numeric(X_all[c], errors="coerce").astype(float)

    keep_mask = ~y_all.isna()
    X = X_all.loc[keep_mask].reset_index(drop=True)
    y = y_all.loc[keep_mask].reset_index(drop=True)

    cat_feature_idx = [X.columns.get_loc(c) for c in cat_all]

    # ---- Walk-forward CV CatBoost ----
    tscv = TimeSeriesSplit(n_splits=5)

    mae_scores, rmse_scores, r2_scores, rpmse_scores = [], [], [], []

    oof_pred = np.full(len(df_main), np.nan)
    idx_map = np.where(keep_mask)[0]

    for fold, (tr_idx, va_idx) in enumerate(tscv.split(X), 1):
        X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
        y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

        train_pool = Pool(X_tr, y_tr, cat_features=cat_feature_idx)
        valid_pool = Pool(X_va, y_va, cat_features=cat_feature_idx)

        model = CatBoostRegressor(
            loss_function="RMSE",
            depth=cb_params["depth"],
            learning_rate=cb_params["learning_rate"],
            l2_leaf_reg=cb_params["l2_leaf_reg"],
            iterations=cb_params["iterations"],
            random_seed=42,
            od_type="Iter",
            od_wait=100,
            verbose=False
        )
        model.fit(train_pool, eval_set=valid_pool, use_best_model=True)

        pred_tr = model.predict(train_pool)
        rmse_train = np.sqrt(mean_squared_error(y_tr, pred_tr))

        pred_va = model.predict(valid_pool)
        oof_pred[idx_map[va_idx]] = pred_va

        mae = mean_absolute_error(y_va, pred_va)
        rmse = np.sqrt(mean_squared_error(y_va, pred_va))
        r2 = r2_score(y_va, pred_va)
        rpmse = (rmse / np.mean(y_va)) * 100

        print(
            f"[{sheet_cluster}] Fold {fold}: "
            f"Train RMSE={rmse_train:.2f} | "
            f"Val RMSE={rmse:.2f} | "
            f"MAE={mae:.2f} | "
            f"RPMSE={rpmse:.2f}% | "
            f"R²={r2:.3f}"
        )

        mae_scores.append(mae)
        rmse_scores.append(rmse)
        r2_scores.append(r2)
        rpmse_scores.append(rpmse)

    print(f"\n=== Hasil Walk-Forward CV (TimeSeriesSplit) untuk {sheet_cluster} ===")
    print(f"MAE   : {np.mean(mae_scores):.2f} ± {np.std(mae_scores):.2f}")
    print(f"RMSE  : {np.mean(rmse_scores):.2f} ± {np.std(rmse_scores):.2f}")
    print(f"RPMSE : {np.mean(rpmse_scores):.2f}% ± {np.std(rpmse_scores):.2f}%")
    print(f"R²    : {np.mean(r2_scores):.3f} ± {np.std(r2_scores):.3f}")

    
    # SHAP: Train 1 model final di seluruh data X,y
    
    print(f"\n[SHAP] Training final CatBoost model untuk SHAP pada sheet {sheet_cluster} ...")

    final_model = CatBoostRegressor(
        loss_function="RMSE",
        depth=cb_params["depth"],
        learning_rate=cb_params["learning_rate"],
        l2_leaf_reg=cb_params["l2_leaf_reg"],
        iterations=cb_params["iterations"],
        random_seed=42,
        od_type="Iter",
        od_wait=100,
        verbose=False
    )

    final_pool = Pool(X, y, cat_features=cat_feature_idx)
    final_model.fit(final_pool)

    shap_values_cat = final_model.get_feature_importance(
        final_pool,
        type="ShapValues"
    )
    shap_values = shap_values_cat[:, :-1]
    base_values = shap_values_cat[:, -1]

    mean_abs_shap = np.abs(shap_values).mean(axis=0)
    shap_importance = pd.DataFrame({
        "feature": X.columns,
        "mean_abs_shap": mean_abs_shap
    }).sort_values("mean_abs_shap", ascending=False)

    print("\n[SHAP] Global feature importance (mean |SHAP|):")
    print(shap_importance)

    # ============================================
    # SAVE SHAP IMPORTANCE → 1 FILE, MULTI-SHEET
    # ============================================
    if os.path.exists(SHAP_FILE):
        mode = "a"
        if_sheet_exists = "replace"
    else:
        mode = "w"
        if_sheet_exists = None

    with pd.ExcelWriter(
        SHAP_FILE,
        engine="openpyxl",
        mode=mode,
        if_sheet_exists=if_sheet_exists
    ) as writer:
        shap_importance.to_excel(writer, sheet_name=sheet_name_out, index=False)

    print(f"[SHAP] Rangkuman SHAP importance tersimpan di file: {SHAP_FILE} | sheet: {sheet_name_out}")

    save_row_shap = False
    if save_row_shap:
        df_shap_row = pd.DataFrame(shap_values, columns=[f"SHAP_{c}" for c in X.columns])
        df_shap_row["base_value"] = base_values
        df_shap_row["y_true"] = y.values

        shap_row_file = f"SHAP_per_row_{sheet_name_out}.xlsx"
        df_shap_row.to_excel(shap_row_file, index=False)
        print(f"[SHAP] SHAP per baris tersimpan di: {shap_row_file}")

    if PLOT_SHAP_SUMMARY:
        try:
            explainer = shap.TreeExplainer(final_model)
            shap_values_for_plot = explainer.shap_values(X)

            shap.summary_plot(
                shap_values_for_plot,
                X,
                feature_names=X.columns,
                show=True
            )
        except Exception as e:
            print(f"[SHAP] Gagal membuat summary_plot: {e}")

    # ============================================
    # SAVE OOF → 1 FILE, MULTI-SHEET
    # ============================================
    df_out = df_main.copy()
    df_out["oof_pred"] = oof_pred

    if price_col not in df_out.columns:
        df_out[price_col] = y_all.values

    cols_to_save = [
        c for c in [CLUSTER_COL, price_col, "Inflation", "GDP_Growth", "oof_pred", "__orig_idx__"]
        if c in df_out.columns
    ]

    # urutkan kembali sesuai urutan baris awal input
    df_save = df_out[cols_to_save].sort_values("__orig_idx__").drop(columns="__orig_idx__")

    if os.path.exists(OOF_FILE):
        mode = "a"
        if_sheet_exists = "replace"
    else:
        mode = "w"
        if_sheet_exists = None

    with pd.ExcelWriter(
        OOF_FILE,
        engine="openpyxl",
        mode=mode,
        if_sheet_exists=if_sheet_exists
    ) as writer:
        df_save.to_excel(writer, sheet_name=sheet_name_out, index=False)

    print(f"\nPrediksi OOF tersimpan di: {OOF_FILE} | sheet: {sheet_name_out}")


In [122]:
if __name__ == "__main__":
    # Load data inflasi & PDRB
    infl_long = load_monthly_inflation(INFLATION_FILE)
    pdrb_growth = load_pdrb_growth(PDRB_FILE)

    # Load Excel utama
    xls = pd.ExcelFile(FILE_CLUSTERED)

    # Loop untuk masing-masing konfigurasi sheet (Monthly & Daily)
    for cfg in SHEET_CONFIGS:
        run_for_sheet(
            xls=xls,
            sheet_cluster=cfg["SHEET_CLUSTER"],
            price_col_name=cfg["PRICE_COL"],
            infl_long=infl_long,
            pdrb_growth=pdrb_growth,
            sheet_name_out=cfg["sheet_name_out"],
        )



================= Proses Monthly_Fixed_clustered =================
[Monthly_Fixed_clustered] Fold 1: Train RMSE=54750.01 | Val RMSE=73954.17 | MAE=50236.89 | RPMSE=29.27% | R²=0.894
[Monthly_Fixed_clustered] Fold 2: Train RMSE=58915.05 | Val RMSE=68417.27 | MAE=47618.57 | RPMSE=29.27% | R²=0.896
[Monthly_Fixed_clustered] Fold 3: Train RMSE=54391.71 | Val RMSE=63105.43 | MAE=41802.57 | RPMSE=26.69% | R²=0.923
[Monthly_Fixed_clustered] Fold 4: Train RMSE=65114.44 | Val RMSE=61113.31 | MAE=43620.23 | RPMSE=25.57% | R²=0.920
[Monthly_Fixed_clustered] Fold 5: Train RMSE=55289.33 | Val RMSE=136939.37 | MAE=60499.96 | RPMSE=56.92% | R²=0.868

=== Hasil Walk-Forward CV (TimeSeriesSplit) untuk Monthly_Fixed_clustered ===
MAE   : 48755.64 ± 6574.71
RMSE  : 80705.91 ± 28469.09
RPMSE : 33.55% ± 11.77%
R²    : 0.900 ± 0.020

[SHAP] Training final CatBoost model untuk SHAP pada sheet Monthly_Fixed_clustered ...

[SHAP] Global feature importance (mean |SHAP|):
                feature  mean_abs_shap


In [66]:
if __name__ == "__main__":
    # Load data inflasi & PDRB
    infl_long = load_monthly_inflation(INFLATION_FILE)
    pdrb_growth = load_pdrb_growth(PDRB_FILE)
    
    # Load Excel utama
    xls = pd.ExcelFile(FILE_CLUSTERED)

    # Loop untuk masing-masing konfigurasi sheet (Monthly & Daily)
    for cfg in SHEET_CONFIGS:
        run_for_sheet(
            xls=xls,
            sheet_cluster=cfg["SHEET_CLUSTER"],
            price_sheet=cfg["PRICE_SHEET"],
            price_col_name=cfg["PRICE_COL"],
            output_file=cfg["OUTPUT_FILE"],
            infl_long=infl_long,
            pdrb_growth=pdrb_growth,
        )


================= Proses Monthly_Fixed_clustered / Monthly_Fixed_CuryUnitPrice =================
[Monthly_Fixed_clustered] Fold 1: Train RMSE=39402.07 | Val RMSE=75605.53 | MAE=48665.33 | RPMSE=29.92% | R²=0.889
[Monthly_Fixed_clustered] Fold 2: Train RMSE=38918.18 | Val RMSE=70459.53 | MAE=43083.04 | RPMSE=30.15% | R²=0.889
[Monthly_Fixed_clustered] Fold 3: Train RMSE=37819.88 | Val RMSE=56336.91 | MAE=36555.17 | RPMSE=23.83% | R²=0.938
[Monthly_Fixed_clustered] Fold 4: Train RMSE=54612.72 | Val RMSE=60556.49 | MAE=42169.52 | RPMSE=25.34% | R²=0.921
[Monthly_Fixed_clustered] Fold 5: Train RMSE=45453.37 | Val RMSE=201341.30 | MAE=69180.15 | RPMSE=83.68% | R²=0.714

=== Hasil Walk-Forward CV (TimeSeriesSplit) untuk Monthly_Fixed_clustered ===
MAE   : 47930.64 ± 11297.54
RMSE  : 92859.96 ± 54672.01
RPMSE : 38.58% ± 22.69%
R²    : 0.870 ± 0.081

[SHAP] Training final CatBoost model untuk SHAP pada sheet Monthly_Fixed_clustered ...

[SHAP] Global feature importance (mean |SHAP|):
        

## visualization

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def _smart_read(path: str) -> pd.DataFrame:
    """
    Helper kecil: baca file berdasarkan ekstensi.
    Sesuaikan kalau semua file kamu pasti Excel / Parquet saja.
    """
    path_lower = path.lower()
    if path_lower.endswith(".parquet"):
        return pd.read_parquet(path)
    elif path_lower.endswith((".xlsx", ".xls")):
        return pd.read_excel(path)
    else:
        # default ke CSV
        return pd.read_csv(path)


def plot_oof_per_unitid(cfg: dict, unit_id: str,
                        id_col: str = "UnitID"):

    freq_name    = cfg["freq_name"]
    price_col    = cfg["PRICE_COL"]              # nama kolom harga ACTUAL di kontrak / target
    oof_file     = cfg["OUTPUT_FILE"]            # <-- isi dengan "catboost_oof_pred_with_cluster...."
    kontrak_file = cfg["KONTRAK_FILE"]           # <-- isi dengan "kontrak_sewa_bersih...."

    # === 1. Baca 2 file ===
    df_oof     = _smart_read(oof_file)
    df_kontrak = _smart_read(kontrak_file)

    # Pastikan RowID ada di kedua file
    if "RowID" not in df_oof.columns:
        raise KeyError("Kolom 'RowID' tidak ada di file catboost_oof_pred_with_cluster.")
    if "RowID" not in df_kontrak.columns:
        raise KeyError("Kolom 'RowID' tidak ada di file kontrak_sewa_bersih.")

    # === 2. Merge berdasarkan RowID ===
    df_base = df_oof.merge(
        df_kontrak,
        on="RowID",
        how="left",
        suffixes=("", "_kontrak")
    )

    # === 3. Pastikan kolom harga ACTUAL benar ===
    #    (harga actual biasanya di kontrak / target)
    if price_col not in df_base.columns:
        cands = [c for c in df_base.columns if "price" in c.lower() or "tarif" in c.lower()]
        if not cands:
            raise KeyError(
                f"Kolom harga actual '{price_col}' tidak ditemukan dan tidak ada kolom mirip 'price/tarif'."
            )
        price_col = cands[0]  # pakai kandidat pertama

    # === 4. Pastikan kolom prediksi ada ===
    if "oof_pred" not in df_base.columns:
        cands_pred = [c for c in df_base.columns if "pred" in c.lower()]
        if not cands_pred:
            raise KeyError("Kolom 'oof_pred' tidak ditemukan di file catboost_oof_pred_with_cluster.")
        pred_col = cands_pred[0]
    else:
        pred_col = "oof_pred"

    # === 5. Isi kolom Actual & Predicted ===
    df_base["Actual"]    = pd.to_numeric(df_base[price_col], errors="coerce")
    df_base["Predicted"] = pd.to_numeric(df_base[pred_col],  errors="coerce")

    # === 6. Filter hanya baris untuk UnitID yang diminta ===
    if id_col not in df_base.columns:
        raise KeyError(f"Kolom ID '{id_col}' tidak ada di hasil merge (cek file kontrak_sewa_bersih).")

    df_base = df_base[df_base[id_col] == unit_id].copy()

    if df_base.empty:
        print(f"Tidak ada data untuk {id_col} = {unit_id}")
        return None

    # Pastikan LeaseStartDate dalam bentuk datetime
    if "LeaseStartDate" not in df_base.columns:
        raise KeyError("Kolom 'LeaseStartDate' tidak ada di hasil merge (cek kontrak_sewa_bersih).")

    df_base["LeaseStartDate"] = pd.to_datetime(df_base["LeaseStartDate"])

    # Sort berdasarkan tanggal
    df_base = df_base.sort_values("LeaseStartDate")

    # === 7. Hitung SMA untuk Predicted di seluruh periode ===
    SMOOTH_WINDOW = 7   # smoothing Predicted
    df_base["Predicted_SMA"] = (
        df_base["Predicted"]
        .rolling(window=SMOOTH_WINDOW, min_periods=1)
        .mean()
    )

    # === 8. Plot dalam 1 grafik ===
    plt.figure(figsize=(12, 4))

    # Raw data Actual & Predicted
    plt.plot(
        df_base["LeaseStartDate"], df_base["Actual"],
        alpha=0.6, linewidth=2.0, label="Actual (raw)"
    )
    plt.plot(
        df_base["LeaseStartDate"], df_base["Predicted"],
        alpha=0.6, linewidth=2.0, label="Predicted (raw)"
    )



    plt.title(f"{freq_name} — {id_col}={unit_id} — Actual vs Predicted")
    plt.xlabel("Lease Start Date")
    plt.ylabel(price_col)
    plt.legend()
    plt.tight_layout()

    # === 9. Simpan gambar ===
    safe_unit = str(unit_id).replace("/", "_").replace("\\", "_")
    out_file = f"{freq_name}_{id_col}_{safe_unit}_actual_vs_predicted.png"

    if os.path.exists(out_file):
        os.remove(out_file)

    plt.savefig(out_file, dpi=200, bbox_inches="tight")
    print(f"Gambar disimpan: {out_file}")

    plt.show()

    return out_file


## Hyperparameter tuning

In [68]:
import optuna

C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def clean_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = df.columns.astype(str).str.strip()
    return df


def parse_price_column(df: pd.DataFrame, col: str) -> pd.Series:
    s = df[col].astype(str).str.strip()
    s = s.str.replace(",", ".", regex=False)
    return pd.to_numeric(s, errors="coerce")


# Load Inflasi Bulanan 
def load_monthly_inflation(path: str) -> pd.DataFrame:
    df_raw = pd.read_csv(path, sep=';', engine='python')
    df_raw = clean_columns(df_raw)

    # baris pertama (index 0) berisi label tahun untuk kolom-kolom lain
    year_labels = df_raw.iloc[0, 1:]

    month_map = {
        "Januari": 1, "Februari": 2, "Maret": 3, "April": 4, "Mei": 5,
        "Juni": 6, "Juli": 7, "Agustus": 8, "September": 9, "Oktober": 10,
        "November": 11, "Desember": 12
    }

    rows = []
    # data bulan mulai di baris index 3 (sesuai file inflasi)
    for i in range(3, len(df_raw)):
        month_name = df_raw.iloc[i, 0]
        if not isinstance(month_name, str):
            continue
        month_name = month_name.strip()
        if month_name == "" or "sumber" in month_name.lower():
            continue

        m = month_map.get(month_name)
        if m is None:
            continue

        for col in df_raw.columns[1:]:
            year_val = year_labels[col]
            try:
                y = int(str(year_val))
            except Exception:
                continue

            val = df_raw.at[i, col]
            if isinstance(val, str):
                val = val.replace(",", ".")
            try:
                infl_val = float(val)
            except Exception:
                continue

            rows.append({
                "LeaseYearStart": y,
                "LeaseMonthStart": m,
                "Inflation": infl_val
            })

    infl_long = pd.DataFrame(rows)

    infl_long = (
        infl_long
        .groupby(["LeaseYearStart", "LeaseMonthStart"], as_index=False)["Inflation"]
        .mean()
    )

    return infl_long


# Load Pertumbuhan PDRB 
def load_pdrb_growth(path: str) -> pd.DataFrame:
    import pandas as pd

    # Paksa pakai delimiter ';'
    df_raw = pd.read_csv(
        path,
        sep=';',            
        header=None,
        encoding='utf-8-sig'  
    )


    # --- kolom 0: tahun ---
    col_year_raw = df_raw.iloc[:, 0].astype(str).str.strip()
    col_year = pd.to_numeric(col_year_raw, errors="coerce")

    # --- kolom 1: pertumbuhan ---
    col_growth_raw = df_raw.iloc[:, 1].astype(str).str.strip()
    col_growth_clean = (
        col_growth_raw
        .str.replace("%", "", regex=False)
        .str.replace(" ", "", regex=False)
        .str.replace(",", ".", regex=False)
    )
    col_growth = pd.to_numeric(col_growth_clean, errors="coerce")

    mask = ~col_year.isna() & ~col_growth.isna()

    pdrb = pd.DataFrame({
        "LeaseYearStart": col_year[mask].astype("Int64"),
        "GDP_Growth": col_growth[mask]
    }).reset_index(drop=True)

    return pdrb


# Fungsi utama per sheet

def run_for_sheet(
    xls: pd.ExcelFile,
    sheet_cluster: str,
    price_sheet: str,
    price_col_name: str,
    output_file: str,
    infl_long: pd.DataFrame,
    pdrb_growth: pd.DataFrame,
):
    print(f"\n================= Proses {sheet_cluster} / {price_sheet} =================")

    if sheet_cluster not in xls.sheet_names:
        raise KeyError(
            f"Sheet cluster '{sheet_cluster}' tidak ditemukan di {FILE_CLUSTERED}."
        )

    try:
        df_main = clean_columns(pd.read_excel(xls, sheet_name=sheet_cluster))
    except ValueError:
        df_main = clean_columns(pd.read_excel(xls, sheet_name=0))

    if CLUSTER_COL not in df_main.columns:
        raise KeyError(
            f"Kolom cluster '{CLUSTER_COL}' tidak ditemukan di sheet '{sheet_cluster}'. "
            "Pastikan hasil clustering sudah disimpan sebagai kolom tersebut."
        )

    # ---- merge Inflasi dan PDRB ----
    if {"LeaseYearStart", "LeaseMonthStart"}.issubset(df_main.columns):
        df_main = df_main.merge(
            infl_long,
            on=["LeaseYearStart", "LeaseMonthStart"],
            how="left"
        )

    if "LeaseYearStart" in df_main.columns and "LeaseYearStart" in pdrb_growth.columns:
        df_main = df_main.merge(
            pdrb_growth,
            on="LeaseYearStart",
            how="left"
        )
    else:
        print("PERINGATAN: 'LeaseYearStart' tidak ada di pdrb_growth – GDP_Growth tidak dipakai.")

    # ---- load price ----
    if price_sheet not in xls.sheet_names:
        raise KeyError(f"Sheet '{price_sheet}' tidak ditemukan di {FILE_CLUSTERED}.")

    df_price = clean_columns(pd.read_excel(xls, sheet_name=price_sheet))

    price_col = price_col_name
    if price_col not in df_price.columns:
        cands = [c for c in df_price.columns if "price" in c.lower()]
        if not cands:
            raise KeyError(
                f"Kolom target '{price_col}' tidak ditemukan dan tidak ada kolom mirip 'price' di sheet '{price_sheet}'."
            )
        price_col = cands[0]

    if "RowID" in df_price.columns:
        df_price["RowID"] = pd.to_numeric(df_price["RowID"], errors="coerce").astype("Int64")
        s_price = df_price.set_index("RowID")[price_col]
        y_all = pd.Series(df_main.index, index=df_main.index).map(s_price).astype(float)
    else:
        if len(df_price) != len(df_main):
            raise ValueError(
                f"Panjang sheet utama ({len(df_main)}) berbeda dengan sheet '{price_sheet}' ({len(df_price)}). "
                "Tambahkan kolom RowID di price sheet agar bisa dipetakan."
            )
        y_all = (
            pd.to_numeric(df_price[price_col], errors="coerce")
            .astype(float)
            .reset_index(drop=True)
        )

    # ---- urutkan berdasarkan tanggal ----
    need_cols = {"LeaseYearStart", "LeaseMonthStart", "LeaseDayStart"}
    if not need_cols.issubset(df_main.columns):
        raise KeyError(
            "Untuk walk-forward CV dibutuhkan kolom LeaseYearStart, LeaseMonthStart, LeaseDayStart."
        )

    lease_start_date = pd.to_datetime(
        dict(
            year=df_main["LeaseYearStart"],
            month=df_main["LeaseMonthStart"],
            day=df_main["LeaseDayStart"],
        ),
        errors="coerce",
    )

    valid_date_mask = ~lease_start_date.isna()
    df_main = df_main.loc[valid_date_mask].reset_index(drop=True)
    y_all = y_all.loc[valid_date_mask].reset_index(drop=True)
    lease_start_date = lease_start_date.loc[valid_date_mask].reset_index(drop=True)

    order = np.argsort(lease_start_date.values)
    df_main = df_main.iloc[order].reset_index(drop=True)
    y_all = y_all.iloc[order].reset_index(drop=True)
    lease_start_date = lease_start_date.iloc[order].reset_index(drop=True)

    # ---- fitur ----
    all_cat_cols = CATEGORICAL_COLS + [CLUSTER_COL]

    cat_all = [c for c in all_cat_cols if c in df_main.columns]
    num_all = [c for c in NUMERIC_COLS if c in df_main.columns]

    for extra in ["Inflation", "GDP_Growth"]:
        if extra in df_main.columns and extra not in num_all:
            num_all.append(extra)

    feature_cols = cat_all + num_all

    X_all = df_main[feature_cols].copy()

    for c in cat_all:
        X_all[c] = X_all[c].astype(str)
    for c in num_all:
        X_all[c] = pd.to_numeric(X_all[c], errors="coerce").astype(float)

    keep_mask = ~y_all.isna()
    X = X_all.loc[keep_mask].reset_index(drop=True)
    y = y_all.loc[keep_mask].reset_index(drop=True)

    cat_feature_idx = [X.columns.get_loc(c) for c in cat_all]

        
    # HYPERPARAMETER TUNING DENGAN OPTUNA
    
    print("\n-- Hyperparameter tuning dengan Optuna --")

    X_tune = X
    y_tune = y
   
    tscv_tune = TimeSeriesSplit(n_splits=3)
    splits_tune = list(tscv_tune.split(X_tune))

    def objective(trial):

        depth = trial.suggest_int("depth", 4, 10)
        lr = trial.suggest_float("learning_rate", 0.01, 0.15, log=True)
        l2 = trial.suggest_float("l2_leaf_reg", 1.0, 10.0)
        iterations = trial.suggest_int("iterations", 500, 1500)

        rmses = []
        for tr_idx, va_idx in splits_tune:
            X_tr, X_va = X_tune.iloc[tr_idx], X_tune.iloc[va_idx]
            y_tr, y_va = y_tune.iloc[tr_idx], y_tune.iloc[va_idx]

            train_pool = Pool(X_tr, y_tr, cat_features=cat_feature_idx)
            valid_pool = Pool(X_va, y_va, cat_features=cat_feature_idx)

            model = CatBoostRegressor(
                loss_function="RMSE",
                depth=int(depth),
                learning_rate=float(lr),
                l2_leaf_reg=float(l2),
                iterations=int(iterations),
                random_seed=42,
                od_type="Iter",
                od_wait=50,
                verbose=False,
            )
            model.fit(train_pool, eval_set=valid_pool, use_best_model=True)
            pred_va = model.predict(valid_pool)
            rmse = np.sqrt(mean_squared_error(y_va, pred_va))
            rmses.append(rmse)

        mean_rmse = float(np.mean(rmses))

        # CETAK PARAMETER + RMSE SETIAP TRIAL
        print(
            f"Optuna trial {trial.number}: "
            f"depth={depth}, lr={lr:.4f}, l2={l2:.2f}, iter={iterations} "
            f"→ mean RMSE={mean_rmse:.4f}"
        )

        return mean_rmse

    study = optuna.create_study(direction="minimize")
    study.optimize(
        objective,
        n_trials=30,            
        show_progress_bar=False
    )

    best_params = study.best_params
    best_params.setdefault("iterations", 1000)

    print(
        f"Best params (Optuna) untuk {sheet_cluster}: "
        f"{best_params} | Best CV RMSE={study.best_value:.4f}"
    )

In [80]:
if __name__ == "__main__":
    # Load data inflasi & PDRB
    infl_long = load_monthly_inflation(INFLATION_FILE)
    pdrb_growth = load_pdrb_growth(PDRB_FILE)

    print("Data PDRB Growth:")
    print(pdrb_growth)

    # Load Excel utama
    xls = pd.ExcelFile(FILE_CLUSTERED)

    # Loop untuk masing-masing konfigurasi sheet (Monthly & Daily)
    for cfg in SHEET_CONFIGS:
        run_for_sheet(
            xls=xls,
            sheet_cluster=cfg["SHEET_CLUSTER"],
            price_sheet=cfg["PRICE_SHEET"],
            price_col_name=cfg["PRICE_COL"],
            output_file=cfg["OUTPUT_FILE"],
            infl_long=infl_long,
            pdrb_growth=pdrb_growth,
        )

Data PDRB Growth:
   LeaseYearStart  GDP_Growth
0            2015        5.97
1            2016        6.00
2            2017        6.13
3            2018        6.19
4            2019        6.09
5            2020       -4.25
6            2021        4.29
7            2022        6.51
8            2023        5.70
9            2024        5.76

================= Proses Monthly_Fixed_clustered / Monthly_Fixed_CuryUnitPrice =================


[I 2025-11-27 13:11:04,677] A new study created in memory with name: no-name-6db78e3a-be16-4d40-bf30-2ba2b187efc6



-- Hyperparameter tuning dengan Optuna --


[I 2025-11-27 13:18:01,038] Trial 0 finished with value: 102462.4455808956 and parameters: {'depth': 7, 'learning_rate': 0.01096322722032367, 'l2_leaf_reg': 5.63175402716522, 'iterations': 1136}. Best is trial 0 with value: 102462.4455808956.


Optuna trial 0: depth=7, lr=0.0110, l2=5.63, iter=1136 → mean RMSE=102462.4456


...

Optuna trial 27: depth=5, lr=0.0280, l2=6.72, iter=1066 → mean RMSE=5312.1635


[I 2025-11-27 16:23:56,445] Trial 28 finished with value: 5223.435406593727 and parameters: {'depth': 4, 'learning_rate': 0.02003066367920022, 'l2_leaf_reg': 2.9125448236612685, 'iterations': 1230}. Best is trial 28 with value: 5223.435406593727.


Optuna trial 28: depth=4, lr=0.0200, l2=2.91, iter=1230 → mean RMSE=5223.4354


[I 2025-11-27 16:25:55,496] Trial 29 finished with value: 5690.073650712849 and parameters: {'depth': 5, 'learning_rate': 0.03190262082487524, 'l2_leaf_reg': 7.778649978852744, 'iterations': 1248}. Best is trial 28 with value: 5223.435406593727.


Optuna trial 29: depth=5, lr=0.0319, l2=7.78, iter=1248 → mean RMSE=5690.0737
Best params (Optuna) untuk Daily_Fixed_clustered: {'depth': 4, 'learning_rate': 0.02003066367920022, 'l2_leaf_reg': 2.9125448236612685, 'iterations': 1230} | Best CV RMSE=5223.4354


In [ ]:
def run_for_sheet(
    xls: pd.ExcelFile,
    sheet_cluster: str,
    price_sheet: str,
    price_col_name: str,
    output_file: str,
):
    print(f"\n================= Proses {sheet_cluster} / {price_sheet} =================")


    if sheet_cluster not in xls.sheet_names:
        raise KeyError(
            f"Sheet cluster '{sheet_cluster}' tidak ditemukan di {FILE_CLUSTERED}."
        )

    try:
        df_main = clean_columns(pd.read_excel(xls, sheet_name=sheet_cluster))
    except ValueError:
        df_main = clean_columns(pd.read_excel(xls, sheet_name=0))

    # pastikan kolom cluster ada
    if CLUSTER_COL not in df_main.columns:
        raise KeyError(
            f"Kolom cluster '{CLUSTER_COL}' tidak ditemukan di sheet '{sheet_cluster}'. "
            "Pastikan hasil clustering sudah disimpan sebagai kolom tersebut."
        )

    if price_sheet not in xls.sheet_names:
        raise KeyError(f"Sheet '{price_sheet}' tidak ditemukan di {FILE_CLUSTERED}.")

    df_price = clean_columns(pd.read_excel(xls, sheet_name=price_sheet))

    # gunakan local variable untuk kolom price
    price_col = price_col_name

    # pastikan kolom harga ada (atau fallback cari yang mengandung 'price')
    if price_col not in df_price.columns:
        cands = [c for c in df_price.columns if "price" in c.lower()]
        if not cands:
            raise KeyError(
                f"Kolom target '{price_col}' tidak ditemukan dan tidak ada kolom mirip 'price' di sheet '{price_sheet}'."
            )
        price_col = cands[0]

    # parsing harga: ganti koma -> titik
    df_price[price_col] = parse_price_column(df_price, price_col)

    # mapping harga ke baris df_main
    if "RowID" in df_price.columns:
        df_price["RowID"] = pd.to_numeric(df_price["RowID"], errors="coerce").astype("Int64")
        s_price = df_price.set_index("RowID")[price_col]
        y_all = pd.Series(df_main.index, index=df_main.index).map(s_price)
    else:
        if len(df_price) != len(df_main):
            raise ValueError(
                f"Panjang sheet utama ({len(df_main)}) berbeda dengan sheet '{price_sheet}' ({len(df_price)}). "
                "Tambahkan kolom RowID di price sheet agar bisa dipetakan."
            )
        y_all = df_price[price_col].reset_index(drop=True)

   
    need_cols = {"LeaseYearStart", "LeaseMonthStart", "LeaseDayStart"}
    if not need_cols.issubset(df_main.columns):
        raise KeyError(
            "Untuk walk-forward CV dibutuhkan kolom LeaseYearStart, LeaseMonthStart, LeaseDayStart."
        )

    lease_start_date = pd.to_datetime(
        dict(
            year=df_main["LeaseYearStart"],
            month=df_main["LeaseMonthStart"],
            day=df_main["LeaseDayStart"],
        ),
        errors="coerce",
    )

    valid_date_mask = ~lease_start_date.isna()
    df_main = df_main.loc[valid_date_mask].reset_index(drop=True)
    y_all   = y_all.loc[valid_date_mask].reset_index(drop=True)
    lease_start_date = lease_start_date.loc[valid_date_mask].reset_index(drop=True)

    order = np.argsort(lease_start_date.values)
    df_main = df_main.iloc[order].reset_index(drop=True)
    y_all   = y_all.iloc[order].reset_index(drop=True)
    lease_start_date = lease_start_date.iloc[order].reset_index(drop=True)

 
    all_cat_cols = CATEGORICAL_COLS + [CLUSTER_COL]

    cat_all = [c for c in all_cat_cols if c in df_main.columns]
    num_all = [c for c in NUMERIC_COLS if c in df_main.columns]
    feature_cols = cat_all + num_all

    X_all = df_main[feature_cols].copy()

    for c in cat_all:
        X_all[c] = X_all[c].astype(str)
    for c in num_all:
        X_all[c] = pd.to_numeric(X_all[c], errors="coerce").astype(float)

    keep_mask = ~y_all.isna()
    X = X_all.loc[keep_mask].reset_index(drop=True)
    y = y_all.loc[keep_mask].reset_index(drop=True)

    if len(X) == 0:
        print("Semua target NaN setelah parsing harga; cek lagi format CuryUnitPrice.")
        df_out = df_main.copy()
        df_out["oof_pred"] = np.nan
        if price_col not in df_out.columns:
            df_out[price_col] = y_all.values
        cols_to_save = [CLUSTER_COL, price_col, "oof_pred"]
        df_out[cols_to_save].to_excel(output_file, index=False)
        return

    cat_feature_idx = [X.columns.get_loc(c) for c in cat_all]

   
    param_grid = {
        "depth": [6, 8, 10],
        "learning_rate": [0.03, 0.05, 0.1],
        "l2_leaf_reg": [1, 3, 5],
    }

    tscv_tune = TimeSeriesSplit(n_splits=3)
    best_rmse = np.inf
    best_params = None

    print("\n-- Hyperparameter tuning --")
    for depth in param_grid["depth"]:
        for lr in param_grid["learning_rate"]:
            for l2 in param_grid["l2_leaf_reg"]:
                rmses = []
                for tr_idx, va_idx in tscv_tune.split(X):
                    X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
                    y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

                    train_pool = Pool(X_tr, y_tr, cat_features=cat_feature_idx)
                    valid_pool = Pool(X_va, y_va, cat_features=cat_feature_idx)

                    model = CatBoostRegressor(
                        loss_function="RMSE",
                        depth=depth,
                        learning_rate=lr,
                        l2_leaf_reg=l2,
                        iterations=2000,
                        random_seed=42,
                        od_type="Iter",
                        od_wait=100,
                        verbose=False,
                    )
                    model.fit(train_pool, eval_set=valid_pool, use_best_model=True)
                    pred_va = model.predict(valid_pool)
                    rmse = np.sqrt(mean_squared_error(y_va, pred_va))
                    rmses.append(rmse)

                mean_rmse = np.mean(rmses)
                print(
                    f"Tuning: depth={depth}, lr={lr}, l2={l2} -> "
                    f"CV RMSE={mean_rmse:.2f}"
                )
                if mean_rmse < best_rmse:
                    best_rmse = mean_rmse
                    best_params = {"depth": depth, "learning_rate": lr, "l2_leaf_reg": l2}

    print(f"Best params: {best_params} | Best CV RMSE={best_rmse:.2f}")


    tscv = TimeSeriesSplit(n_splits=5)

    mae_scores, rmse_scores, r2_scores, rpmse_scores = [], [], [], []

    oof_pred = np.full(len(df_main), np.nan)
    idx_map  = np.where(keep_mask)[0]

    for fold, (tr_idx, va_idx) in enumerate(tscv.split(X), 1):
        X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
        y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

        train_pool = Pool(X_tr, y_tr, cat_features=cat_feature_idx)
        valid_pool = Pool(X_va, y_va, cat_features=cat_feature_idx)

        model = CatBoostRegressor(
            loss_function="RMSE",
            depth=best_params["depth"],
            learning_rate=best_params["learning_rate"],
            l2_leaf_reg=best_params["l2_leaf_reg"],
            iterations=2000,
            random_seed=42,
            od_type="Iter",
            od_wait=100,
            verbose=False,
        )
        model.fit(train_pool, eval_set=valid_pool, use_best_model=True)

        pred_tr = model.predict(train_pool)
        rmse_train = np.sqrt(mean_squared_error(y_tr, pred_tr))

        pred_va = model.predict(valid_pool)
        oof_pred[idx_map[va_idx]] = pred_va

        mae   = mean_absolute_error(y_va, pred_va)
        rmse  = np.sqrt(mean_squared_error(y_va, pred_va))
        r2    = r2_score(y_va, pred_va)
        rpmse = (rmse / np.mean(y_va)) * 100

        print(
            f"[{sheet_cluster}] Fold {fold}: "
            f"Train RMSE={rmse_train:.2f} | "
            f"Val RMSE={rmse:.2f} | "
            f"MAE={mae:.2f} | "
            f"RPMSE={rpmse:.2f}% | "
            f"R²={r2:.3f}"
        )

        mae_scores.append(mae)
        rmse_scores.append(rmse)
        r2_scores.append(r2)
        rpmse_scores.append(rpmse)

    print(f"\n=== Hasil Walk-Forward CV (TimeSeriesSplit) untuk {sheet_cluster} ===")
    print(f"MAE   : {np.mean(mae_scores):.2f} ± {np.std(mae_scores):.2f}")
    print(f"RMSE  : {np.mean(rmse_scores):.2f} ± {np.std(rmse_scores):.2f}")
    print(f"RPMSE : {np.mean(rpmse_scores):.2f}% ± {np.std(rpmse_scores):.2f}%")
    print(f"R²    : {np.mean(r2_scores):.3f} ± {np.std(r2_scores):.3f}")

    # Simpan hasil OOF 
    df_out = df_main.copy()
    df_out["oof_pred"] = oof_pred

    if price_col not in df_out.columns:
        df_out[price_col] = y_all.values

    cols_to_save = [CLUSTER_COL, price_col, "oof_pred"]
    df_out[cols_to_save].to_excel(output_file, index=False)

    print(f"\nPrediksi OOF tersimpan di: {output_file}")
